# 🎯 LEAPS Screener — May 2026+ Expirations

**Conviction picks (bold):** GEV, AMAT, AVGO  
**Full watchlist:** WDC, GEV, STX, LRCX, AMAT, TSM, GE, CMI, KLAC, SNPS, META, UBER, ISRG, MSFT, AMZN, AVGO

Screens for deep-ITM to ATM call LEAPS with ≥90 DTE (May 2026+).  
Evaluates: IV, liquidity, breakeven, leverage ratio, and cost efficiency.


In [82]:
import time
import warnings
import math
from datetime import datetime
from typing import Optional

import numpy as np
import pandas as pd
import yfinance as yf

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from IPython.display import display, Markdown

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 60)
pd.set_option("display.width", 200)
pd.set_option("display.max_rows", 200)

# ── Config ──────────────────────────────────────────────────────
CONVICTION = ["GEV", "AMAT", "AVGO"]

ALL_TICKERS = [
    "WDC",
    "GEV",
    "STX",
    "LRCX",
    "AMAT",
    "TSM",
    "GE",
    "CMI",
    "KLAC",
    "SNPS",
    "META",
    "UBER",
    "ISRG",
    "MSFT",
    "AMZN",
    "AVGO",
]

MIN_DTE = 90  # May 2026+ from today
MONEYNESS_RANGE = (0.80, 1.05)  # 80% to 105% of spot (deep-ITM to slightly OTM)
RATE_LIMIT_SLEEP = 0.35
MIN_OPEN_INTEREST = 20
DELTA_TARGETS = [0.80, 0.70, 0.60]  # typical LEAPS deltas
RISK_FREE_RATE = 0.045  # ~4.5% annualised (T-bill proxy) — used everywhere

# Plot style
pio.renderers.default = "notebook_connected"

print(f"Tickers: {len(ALL_TICKERS)}")
print(f"Conviction: {', '.join(CONVICTION)}")
print(f"Min DTE: {MIN_DTE} days  (targets May 2026+)")

Tickers: 16
Conviction: GEV, AMAT, AVGO
Min DTE: 90 days  (targets May 2026+)


## 1. Fetch Spot Prices & Available LEAPS Expirations


In [66]:
def get_spot(ticker: str) -> Optional[float]:
    """Get current spot price."""
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period="5d")
        if not hist.empty and "Close" in hist.columns:
            return float(hist["Close"].iloc[-1])
        return None
    except Exception:
        return None


def get_leap_expirations(ticker: str, min_dte: int = MIN_DTE) -> list[tuple[str, int]]:
    """Return expirations with DTE >= min_dte."""
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        today = datetime.now().date()
        result = []
        for exp_str in t.options or []:
            try:
                exp_date = datetime.strptime(exp_str, "%Y-%m-%d").date()
                dte = (exp_date - today).days
                if dte >= min_dte:
                    result.append((exp_str, dte))
            except ValueError:
                continue
        return sorted(result, key=lambda x: x[1])
    except Exception:
        return []


# Fetch spots and expirations
spot_map = {}
exp_map = {}
ticker_info = []

for ticker in ALL_TICKERS:
    spot = get_spot(ticker)
    spot_map[ticker] = spot
    exps = get_leap_expirations(ticker) if spot else []
    exp_map[ticker] = exps

    ticker_info.append(
        {
            "Ticker": ticker,
            "Conviction": "⭐" if ticker in CONVICTION else "",
            "Spot": spot,
            "LEAPS Expirations": len(exps),
            "Nearest LEAPS": exps[0][0] if exps else "—",
            "Nearest DTE": exps[0][1] if exps else None,
            "Farthest LEAPS": exps[-1][0] if exps else "—",
            "Farthest DTE": exps[-1][1] if exps else None,
        }
    )

info_df = pd.DataFrame(ticker_info)
display(Markdown("### Spot Prices & LEAPS Availability"))
display(
    info_df.style.format(
        {
            "Spot": "${:,.2f}",
            "Nearest DTE": "{:.0f}",
            "Farthest DTE": "{:.0f}",
        },
        na_rep="—",
    ).set_caption("All tickers — ⭐ = conviction pick")
)

### Spot Prices & LEAPS Availability

,Ticker,Conviction,Spot,LEAPS Expirations,Nearest LEAPS,Nearest DTE,Farthest LEAPS,Farthest DTE
0,WDC,,$282.58,7,2026-06-18,130,2028-01-21,712
1,GEV,⭐,$779.35,8,2026-05-15,96,2028-12-15,1041
2,STX,,$429.32,6,2026-06-18,130,2028-01-21,712
3,LRCX,,$231.01,6,2026-06-18,130,2028-01-21,712
4,AMAT,⭐,$322.51,10,2026-05-15,96,2028-01-21,712
5,TSM,,$348.85,10,2026-05-15,96,2028-12-15,1041
6,GE,,$321.00,6,2026-05-15,96,2028-01-21,712
7,CMI,,$577.73,2,2026-06-18,130,2026-09-18,222
8,KLAC,,"$1,442.95",7,2026-05-15,96,2028-01-21,712
9,SNPS,,$426.88,5,2026-06-18,130,2028-01-21,712


## 2. Fetch LEAPS Chains & Build Comparison Table

For each ticker, pull call chains at available LEAPS expirations.  
Filter to strikes between 80–105% of spot (deep-ITM to slightly OTM).  
Compute breakeven, leverage ratio, and extrinsic value %.


In [67]:
def safe_float(v):
    try:
        return float(v) if v is not None else np.nan
    except Exception:
        return np.nan


def fetch_leaps_calls(ticker: str, exp_date: str, spot: float) -> pd.DataFrame:
    """Fetch call chain and enrich with LEAPS metrics."""
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        chain = t.option_chain(exp_date)
        df = chain.calls.copy()
        if df.empty:
            return df

        exp_dt = datetime.strptime(exp_date, "%Y-%m-%d").date()
        dte = (exp_dt - datetime.now().date()).days

        df["ticker"] = ticker
        df["expiration"] = exp_date
        df["dte"] = dte
        df["spot"] = spot
        df["mid"] = (df["bid"] + df["ask"]) / 2
        df.loc[df["mid"] <= 0, "mid"] = df["lastPrice"]
        df["moneyness"] = df["strike"] / spot
        df["spread"] = df["ask"] - df["bid"]
        df["spread_pct"] = np.where(df["mid"] > 0, df["spread"] / df["mid"], np.nan)

        # LEAPS-specific metrics
        df["intrinsic"] = np.maximum(spot - df["strike"], 0)
        df["extrinsic"] = df["mid"] - df["intrinsic"]
        df["extrinsic_pct"] = np.where(
            df["mid"] > 0, df["extrinsic"] / df["mid"], np.nan
        )
        df["breakeven"] = df["strike"] + df["mid"]
        df["breakeven_pct"] = (df["breakeven"] - spot) / spot
        df["leverage"] = np.where(df["mid"] > 0, spot / df["mid"], np.nan)
        df["cost_per_contract"] = df["mid"] * 100
        df["iv"] = df["impliedVolatility"]

        # Filter to LEAPS-relevant range
        mask = (
            (df["moneyness"] >= MONEYNESS_RANGE[0])
            & (df["moneyness"] <= MONEYNESS_RANGE[1])
            & (df["mid"] > 0)
        )
        return df[mask].copy()
    except Exception as e:
        print(f"  ⚠ {ticker} {exp_date}: {e}")
        return pd.DataFrame()


# Pull chains for ALL tickers — pick the 2 nearest LEAPS expirations per ticker
all_leaps = []
skipped = []

for ticker in ALL_TICKERS:
    spot = spot_map.get(ticker)
    exps = exp_map.get(ticker, [])
    if not spot or not exps:
        skipped.append(ticker)
        continue

    # Take up to 3 expirations to give options across time horizons
    selected_exps = exps[:3]
    print(f"📊 {ticker} (${spot:.2f}) — pulling {len(selected_exps)} expirations...")

    for exp_date, dte in selected_exps:
        df = fetch_leaps_calls(ticker, exp_date, spot)
        if not df.empty:
            all_leaps.append(df)

if skipped:
    print(f"\n⚠ Skipped (no spot/expirations): {skipped}")

leaps_df = pd.concat(all_leaps, ignore_index=True) if all_leaps else pd.DataFrame()
print(f"\n✅ Total LEAPS options fetched: {len(leaps_df)}")
print(
    f"   Tickers with data: {leaps_df['ticker'].nunique() if not leaps_df.empty else 0}"
)

📊 WDC ($282.58) — pulling 3 expirations...
📊 GEV ($779.35) — pulling 3 expirations...
📊 STX ($429.32) — pulling 3 expirations...
📊 LRCX ($231.01) — pulling 3 expirations...
📊 AMAT ($322.51) — pulling 3 expirations...
📊 TSM ($348.85) — pulling 3 expirations...
📊 GE ($321.00) — pulling 3 expirations...
📊 CMI ($577.73) — pulling 2 expirations...
📊 KLAC ($1442.95) — pulling 3 expirations...
📊 SNPS ($426.88) — pulling 3 expirations...
📊 META ($661.46) — pulling 3 expirations...
📊 UBER ($74.77) — pulling 3 expirations...
📊 ISRG ($488.15) — pulling 3 expirations...
📊 MSFT ($401.14) — pulling 3 expirations...
📊 AMZN ($210.32) — pulling 3 expirations...
📊 AVGO ($332.92) — pulling 3 expirations...

✅ Total LEAPS options fetched: 536
   Tickers with data: 16


## 3. Best LEAPS Picks per Ticker

For each ticker, find the "sweet spot" LEAPS: ~80–90% moneyness (10–20% ITM) with reasonable liquidity.  
These give stock-like delta exposure at a fraction of the cost.


In [68]:
if leaps_df.empty:
    display(Markdown("❌ No LEAPS data available."))
else:
    # For each ticker × expiration, pick the best option at ~0.85 moneyness (15% ITM)
    # and also show ATM for comparison
    pick_rows = []

    for ticker in ALL_TICKERS:
        tdf = leaps_df[leaps_df["ticker"] == ticker]
        if tdf.empty:
            continue
        spot = spot_map[ticker]

        for exp_date in tdf["expiration"].unique():
            edf = tdf[tdf["expiration"] == exp_date].copy()
            dte = edf["dte"].iloc[0]

            # Deep ITM pick (~80-85% moneyness for high delta)
            deep_itm = edf[(edf["moneyness"] >= 0.78) & (edf["moneyness"] <= 0.87)]
            if not deep_itm.empty:
                best = deep_itm.loc[(deep_itm["moneyness"] - 0.82).abs().idxmin()]
                pick_rows.append(
                    {
                        "Ticker": ticker,
                        "Conviction": "⭐" if ticker in CONVICTION else "",
                        "Type": "Deep ITM",
                        "Expiration": exp_date,
                        "DTE": dte,
                        "Strike": best["strike"],
                        "Moneyness": best["moneyness"],
                        "Bid": best["bid"],
                        "Ask": best["ask"],
                        "Mid": best["mid"],
                        "IV": best["iv"],
                        "Intrinsic": best["intrinsic"],
                        "Extrinsic": best["extrinsic"],
                        "Extrinsic %": best["extrinsic_pct"],
                        "Breakeven": best["breakeven"],
                        "Breakeven %": best["breakeven_pct"],
                        "Leverage": best["leverage"],
                        "Cost/Contract": best["cost_per_contract"],
                        "OI": safe_float(best.get("openInterest")),
                        "Volume": safe_float(best.get("volume")),
                        "Spread %": best["spread_pct"],
                        "Spot": spot,
                    }
                )

            # Moderate ITM pick (~90% moneyness)
            mod_itm = edf[(edf["moneyness"] >= 0.87) & (edf["moneyness"] <= 0.93)]
            if not mod_itm.empty:
                best = mod_itm.loc[(mod_itm["moneyness"] - 0.90).abs().idxmin()]
                pick_rows.append(
                    {
                        "Ticker": ticker,
                        "Conviction": "⭐" if ticker in CONVICTION else "",
                        "Type": "ITM ~90%",
                        "Expiration": exp_date,
                        "DTE": dte,
                        "Strike": best["strike"],
                        "Moneyness": best["moneyness"],
                        "Bid": best["bid"],
                        "Ask": best["ask"],
                        "Mid": best["mid"],
                        "IV": best["iv"],
                        "Intrinsic": best["intrinsic"],
                        "Extrinsic": best["extrinsic"],
                        "Extrinsic %": best["extrinsic_pct"],
                        "Breakeven": best["breakeven"],
                        "Breakeven %": best["breakeven_pct"],
                        "Leverage": best["leverage"],
                        "Cost/Contract": best["cost_per_contract"],
                        "OI": safe_float(best.get("openInterest")),
                        "Volume": safe_float(best.get("volume")),
                        "Spread %": best["spread_pct"],
                        "Spot": spot,
                    }
                )

            # ATM pick (~100% moneyness)
            atm = edf[(edf["moneyness"] >= 0.97) & (edf["moneyness"] <= 1.03)]
            if not atm.empty:
                best = atm.loc[(atm["moneyness"] - 1.0).abs().idxmin()]
                pick_rows.append(
                    {
                        "Ticker": ticker,
                        "Conviction": "⭐" if ticker in CONVICTION else "",
                        "Type": "ATM",
                        "Expiration": exp_date,
                        "DTE": dte,
                        "Strike": best["strike"],
                        "Moneyness": best["moneyness"],
                        "Bid": best["bid"],
                        "Ask": best["ask"],
                        "Mid": best["mid"],
                        "IV": best["iv"],
                        "Intrinsic": best["intrinsic"],
                        "Extrinsic": best["extrinsic"],
                        "Extrinsic %": best["extrinsic_pct"],
                        "Breakeven": best["breakeven"],
                        "Breakeven %": best["breakeven_pct"],
                        "Leverage": best["leverage"],
                        "Cost/Contract": best["cost_per_contract"],
                        "OI": safe_float(best.get("openInterest")),
                        "Volume": safe_float(best.get("volume")),
                        "Spread %": best["spread_pct"],
                        "Spot": spot,
                    }
                )

    picks_df = pd.DataFrame(pick_rows)

    display(Markdown("### 🏆 LEAPS Picks — All Tickers"))
    display(
        Markdown(
            "*Deep ITM = highest delta / stock replacement | ITM ~90% = balance of delta + leverage | ATM = max leverage*"
        )
    )

    fmt = {
        "Strike": "${:,.0f}",
        "Bid": "${:,.2f}",
        "Ask": "${:,.2f}",
        "Mid": "${:,.2f}",
        "IV": "{:.1%}",
        "Intrinsic": "${:,.2f}",
        "Extrinsic": "${:,.2f}",
        "Extrinsic %": "{:.1%}",
        "Breakeven": "${:,.2f}",
        "Breakeven %": "{:+.1%}",
        "Leverage": "{:.1f}x",
        "Cost/Contract": "${:,.0f}",
        "OI": "{:,.0f}",
        "Volume": "{:,.0f}",
        "Spread %": "{:.1%}",
        "Spot": "${:,.2f}",
        "Moneyness": "{:.1%}",
    }
    display(
        picks_df.style.format(fmt, na_rep="—").set_caption(
            "LEAPS Call Options — May 2026+ Expirations"
        )
    )

### 🏆 LEAPS Picks — All Tickers

*Deep ITM = highest delta / stock replacement | ITM ~90% = balance of delta + leverage | ATM = max leverage*

,Ticker,Conviction,Type,Expiration,DTE,Strike,Moneyness,Bid,Ask,Mid,IV,Intrinsic,Extrinsic,Extrinsic %,Breakeven,Breakeven %,Leverage,Cost/Contract,OI,Volume,Spread %,Spot
0,WDC,,Deep ITM,2026-06-18,130,$230,81.4%,$79.55,$85.50,$82.53,86.2%,$52.58,$29.95,36.3%,$312.52,+10.6%,3.4x,"$8,252",430,3,7.2%,$282.58
1,WDC,,ITM ~90%,2026-06-18,130,$250,88.5%,$68.00,$74.85,$71.42,85.2%,$32.58,$38.85,54.4%,$321.43,+13.7%,4.0x,"$7,142",545,12,9.6%,$282.58
2,WDC,,ATM,2026-06-18,130,$280,99.1%,$55.10,$60.30,$57.70,84.8%,$2.58,$55.12,95.5%,$337.70,+19.5%,4.9x,"$5,770",326,23,9.0%,$282.58
3,WDC,,Deep ITM,2026-07-17,159,$230,81.4%,$84.00,$89.50,$86.75,85.0%,$52.58,$34.17,39.4%,$316.75,+12.1%,3.3x,"$8,675",86,12,6.3%,$282.58
4,WDC,,ITM ~90%,2026-07-17,159,$250,88.5%,$73.75,$79.00,$76.38,84.6%,$32.58,$43.80,57.3%,$326.38,+15.5%,3.7x,"$7,638",139,37,6.9%,$282.58
5,WDC,,ATM,2026-07-17,159,$280,99.1%,$59.90,$65.55,$62.72,83.7%,$2.58,$60.15,95.9%,$342.73,+21.3%,4.5x,"$6,272",82,21,9.0%,$282.58
6,GEV,⭐,Deep ITM,2026-05-15,96,$640,82.1%,$169.70,$174.00,$171.85,58.3%,$139.35,$32.50,18.9%,$811.85,+4.2%,4.5x,"$17,185",60,1,2.5%,$779.35
7,GEV,⭐,ITM ~90%,2026-05-15,96,$700,89.8%,$126.30,$131.90,$129.10,55.1%,$79.35,$49.75,38.5%,$829.10,+6.4%,6.0x,"$12,910",135,4,4.3%,$779.35
8,GEV,⭐,ATM,2026-05-15,96,$780,100.1%,$83.20,$85.90,$84.55,53.1%,$0.00,$84.55,100.0%,$864.55,+10.9%,9.2x,"$8,455",150,9,3.2%,$779.35
9,GEV,⭐,Deep ITM,2026-06-18,130,$640,82.1%,$181.70,$185.70,$183.70,58.7%,$139.35,$44.35,24.1%,$823.70,+5.7%,4.2x,"$18,370",33,4,2.2%,$779.35


## 4. Conviction Picks Deep Dive — GEV, AMAT, AVGO

Detailed comparison of your three conviction names across strike levels and expirations.  
Key question: **what's the cheapest way to get long-dated exposure?**


In [69]:
if leaps_df.empty:
    display(Markdown("❌ No data."))
else:
    for ticker in CONVICTION:
        tdf = leaps_df[leaps_df["ticker"] == ticker].copy()
        spot = spot_map.get(ticker)
        if tdf.empty or spot is None:
            display(Markdown(f"### ⚠ {ticker} — no LEAPS data"))
            continue

        display(Markdown(f"---\n### ⭐ {ticker}  —  Spot: ${spot:,.2f}"))

        # Show full chain filtered to interesting range for nearest LEAPS expiration
        nearest_exp = tdf["expiration"].min()
        nearest_dte = tdf[tdf["expiration"] == nearest_exp]["dte"].iloc[0]

        chain = tdf[tdf["expiration"] == nearest_exp].sort_values("strike")

        display(Markdown(f"**Nearest LEAPS:** {nearest_exp} ({nearest_dte} DTE)"))

        show_cols = [
            "strike",
            "moneyness",
            "bid",
            "ask",
            "mid",
            "iv",
            "intrinsic",
            "extrinsic",
            "extrinsic_pct",
            "breakeven",
            "breakeven_pct",
            "leverage",
            "cost_per_contract",
            "openInterest",
            "volume",
            "spread_pct",
        ]
        available_cols = [c for c in show_cols if c in chain.columns]
        display(
            chain[available_cols]
            .style.format(
                {
                    "strike": "${:,.0f}",
                    "moneyness": "{:.1%}",
                    "bid": "${:,.2f}",
                    "ask": "${:,.2f}",
                    "mid": "${:,.2f}",
                    "iv": "{:.1%}",
                    "intrinsic": "${:,.2f}",
                    "extrinsic": "${:,.2f}",
                    "extrinsic_pct": "{:.1%}",
                    "breakeven": "${:,.2f}",
                    "breakeven_pct": "{:+.1%}",
                    "leverage": "{:.1f}x",
                    "cost_per_contract": "${:,.0f}",
                    "openInterest": "{:,.0f}",
                    "volume": "{:,.0f}",
                    "spread_pct": "{:.1%}",
                },
                na_rep="—",
            )
            .set_caption(f"{ticker} — {nearest_exp} LEAPS Calls")
        )

        # Scenario analysis: what happens if stock moves ±10%, ±20%
        display(Markdown(f"\n**Scenario P&L (at expiration) — {ticker}**"))
        scenarios = [-0.20, -0.10, 0.0, +0.10, +0.20, +0.30]
        scenario_rows = []
        # Pick 3 representative strikes
        for moneyness_target in [0.85, 0.90, 1.00]:
            candidates = chain[(chain["moneyness"] - moneyness_target).abs() < 0.05]
            if candidates.empty:
                continue
            opt = candidates.loc[
                (candidates["moneyness"] - moneyness_target).abs().idxmin()
            ]
            strike = opt["strike"]
            cost = opt["mid"]
            for move in scenarios:
                future_spot = spot * (1 + move)
                payout = max(future_spot - strike, 0)
                pnl = payout - cost
                pnl_pct = pnl / cost if cost > 0 else np.nan
                scenario_rows.append(
                    {
                        "Strike": strike,
                        "Moneyness": opt["moneyness"],
                        "Cost": cost,
                        "Stock Move": move,
                        "Future Spot": future_spot,
                        "Payout": payout,
                        "P&L": pnl,
                        "P&L %": pnl_pct,
                    }
                )

        if scenario_rows:
            sdf = pd.DataFrame(scenario_rows)
            pivot = sdf.pivot_table(
                index=["Strike", "Moneyness", "Cost"],
                columns="Stock Move",
                values="P&L %",
            )
            pivot.columns = [f"{c:+.0%}" for c in pivot.columns]
            display(
                pivot.style.format("{:+.0%}", na_rep="—")
                .background_gradient(cmap="RdYlGn", axis=None, vmin=-1, vmax=2)
                .set_caption(f"{ticker} — LEAPS P&L % at Expiration by Stock Move")
            )

---
### ⭐ GEV  —  Spot: $779.35

**Nearest LEAPS:** 2026-05-15 (96 DTE)

,strike,moneyness,bid,ask,mid,iv,intrinsic,extrinsic,extrinsic_pct,breakeven,breakeven_pct,leverage,cost_per_contract,openInterest,volume,spread_pct
14,$630,80.8%,$177.80,$181.60,$179.70,59.1%,$149.35,$30.35,16.9%,$809.70,+3.9%,4.3x,"$17,970",26,1,2.1%
15,$640,82.1%,$169.70,$174.00,$171.85,58.3%,$139.35,$32.50,18.9%,$811.85,+4.2%,4.5x,"$17,185",60,1,2.5%
16,$650,83.4%,$162.10,$167.00,$164.55,57.9%,$129.35,$35.20,21.4%,$814.55,+4.5%,4.7x,"$16,455",81,3,3.0%
17,$660,84.7%,$154.00,$159.20,$156.60,56.9%,$119.35,$37.25,23.8%,$816.60,+4.8%,5.0x,"$15,660",47,1,3.3%
18,$670,86.0%,$147.30,$152.00,$149.65,56.6%,$109.35,$40.30,26.9%,$819.65,+5.2%,5.2x,"$14,965",53,6,3.1%
19,$680,87.3%,$141.10,$145.80,$143.45,56.7%,$99.35,$44.10,30.7%,$823.45,+5.7%,5.4x,"$14,345",64,13,3.3%
20,$690,88.5%,$133.00,$137.90,$135.45,55.4%,$89.35,$46.10,34.0%,$825.45,+5.9%,5.8x,"$13,545",117,5,3.6%
21,$700,89.8%,$126.30,$131.90,$129.10,55.1%,$79.35,$49.75,38.5%,$829.10,+6.4%,6.0x,"$12,910",135,4,4.3%
22,$710,91.1%,$120.10,$125.00,$122.55,54.6%,$69.35,$53.20,43.4%,$832.55,+6.8%,6.4x,"$12,255",80,3,4.0%
23,$720,92.4%,$114.20,$120.00,$117.10,54.8%,$59.35,$57.75,49.3%,$837.10,+7.4%,6.7x,"$11,710",165,1,5.0%



**Scenario P&L (at expiration) — GEV**

,,,-20%,-10%,+0%,+10%,+20%,+30%
Strike,Moneyness,Cost,,,,,,
660.000000,0.846860,156.600000,-100%,-74%,-24%,+26%,+76%,+126%
700.000000,0.898184,129.100000,-100%,-99%,-39%,+22%,+82%,+143%
780.000000,1.000834,84.550000,-100%,-100%,-100%,-9%,+84%,+176%


...

---
### ⭐ AVGO  —  Spot: $332.92

**Nearest LEAPS:** 2026-05-15 (96 DTE)

,strike,moneyness,bid,ask,mid,iv,intrinsic,extrinsic,extrinsic_pct,breakeven,breakeven_pct,leverage,cost_per_contract,openInterest,volume,spread_pct
512,$270,81.1%,$76.15,$78.70,$77.43,61.7%,$62.92,$14.50,18.7%,$347.43,+4.4%,4.3x,"$7,743",109,21,3.3%
513,$280,84.1%,$69.05,$71.10,$70.07,60.5%,$52.92,$17.15,24.5%,$350.07,+5.2%,4.8x,"$7,007",858,5,2.9%
514,$290,87.1%,$61.70,$64.10,$62.90,59.0%,$42.92,$19.98,31.8%,$352.90,+6.0%,5.3x,"$6,290",546,9,3.8%
515,$300,90.1%,$55.60,$57.50,$56.55,58.4%,$32.92,$23.63,41.8%,$356.55,+7.1%,5.9x,"$5,655",798,94,3.4%
516,$310,93.1%,$48.90,$51.30,$50.10,57.0%,$22.92,$27.18,54.3%,$360.10,+8.2%,6.6x,"$5,010","1,162",99,4.8%
517,$320,96.1%,$43.85,$44.95,$44.40,56.2%,$12.92,$31.48,70.9%,$364.40,+9.5%,7.5x,"$4,440","1,479",393,2.5%
518,$330,99.1%,$38.30,$39.25,$38.77,54.9%,$2.92,$35.85,92.5%,$368.77,+10.8%,8.6x,"$3,878","1,152",287,2.5%
519,$340,102.1%,$33.80,$34.45,$34.12,54.5%,$0.00,$34.12,100.0%,$374.12,+12.4%,9.8x,"$3,412","2,644",290,1.9%



**Scenario P&L (at expiration) — AVGO**

,,,-20%,-10%,+0%,+10%,+20%,+30%
Strike,Moneyness,Cost,,,,,,
280.000000,0.841043,70.075000,-100%,-72%,-24%,+23%,+71%,+118%
300.000000,0.901117,56.550000,-100%,-100%,-42%,+17%,+76%,+135%
330.000000,0.991229,38.775000,-100%,-100%,-92%,-7%,+79%,+165%


## 5. Visualizations — IV Smiles & Cost Comparison


In [70]:
if leaps_df.empty:
    display(Markdown("❌ No data."))
else:
    # ── IV Smile for conviction picks ──
    conv_df = leaps_df[leaps_df["ticker"].isin(CONVICTION)].copy()
    if not conv_df.empty:
        # Use nearest expiration per ticker
        nearest = conv_df.groupby("ticker")["expiration"].min().reset_index()
        nearest.columns = ["ticker", "nearest_exp"]
        conv_nearest = conv_df.merge(nearest, on="ticker")
        conv_nearest = conv_nearest[
            conv_nearest["expiration"] == conv_nearest["nearest_exp"]
        ]

        fig = px.line(
            conv_nearest,
            x="moneyness",
            y="iv",
            color="ticker",
            title="LEAPS IV Smile — Conviction Picks (Nearest Expiration)",
            labels={
                "moneyness": "Moneyness (Strike / Spot)",
                "iv": "Implied Volatility",
            },
            markers=True,
        )
        fig.add_vline(x=1.0, line_dash="dash", line_color="gray", annotation_text="ATM")
        fig.update_layout(yaxis_tickformat=".0%", xaxis_tickformat=".0%")
        fig.show()

    # ── Cost per contract comparison (ITM ~90%) ──
    if not picks_df.empty:
        itm_picks = picks_df[picks_df["Type"] == "ITM ~90%"].copy()
        if not itm_picks.empty:
            # Take nearest expiration per ticker
            itm_nearest = (
                itm_picks.sort_values("DTE").groupby("Ticker").first().reset_index()
            )
            itm_nearest["is_conviction"] = itm_nearest["Ticker"].isin(CONVICTION)
            itm_nearest = itm_nearest.sort_values("Cost/Contract")

            fig2 = px.bar(
                itm_nearest,
                x="Ticker",
                y="Cost/Contract",
                color="is_conviction",
                color_discrete_map={True: "#1a7431", False: "#4C6E91"},
                title="LEAPS Cost per Contract — ITM ~90% Strike (Nearest Expiration)",
                labels={
                    "Cost/Contract": "Cost per Contract ($)",
                    "is_conviction": "Conviction",
                },
                text="Cost/Contract",
            )
            fig2.update_traces(texttemplate="$%{text:,.0f}", textposition="outside")
            fig2.update_layout(
                showlegend=True, yaxis_tickprefix="$", yaxis_tickformat=","
            )
            fig2.show()

    # ── Breakeven % comparison ──
    if not picks_df.empty:
        itm_picks = picks_df[picks_df["Type"] == "ITM ~90%"].copy()
        if not itm_picks.empty:
            itm_nearest = (
                itm_picks.sort_values("DTE").groupby("Ticker").first().reset_index()
            )
            itm_nearest["is_conviction"] = itm_nearest["Ticker"].isin(CONVICTION)
            itm_nearest = itm_nearest.sort_values("Breakeven %")

            fig3 = px.bar(
                itm_nearest,
                x="Ticker",
                y="Breakeven %",
                color="is_conviction",
                color_discrete_map={True: "#1a7431", False: "#4C6E91"},
                title="LEAPS Breakeven % Above Spot — ITM ~90% (Nearest Expiration)",
                labels={"Breakeven %": "Breakeven Above Spot (%)"},
                text="Breakeven %",
            )
            fig3.update_traces(texttemplate="%{text:+.1%}", textposition="outside")
            fig3.update_layout(showlegend=True, yaxis_tickformat="+.1%")
            fig3.show()

    # ── Leverage ratio comparison ──
    if not picks_df.empty:
        for opt_type in ["Deep ITM", "ITM ~90%", "ATM"]:
            type_picks = picks_df[picks_df["Type"] == opt_type].copy()
            if type_picks.empty:
                continue
            type_nearest = (
                type_picks.sort_values("DTE").groupby("Ticker").first().reset_index()
            )
            type_nearest["is_conviction"] = type_nearest["Ticker"].isin(CONVICTION)
            type_nearest = type_nearest.sort_values("Leverage", ascending=False)

            fig4 = px.bar(
                type_nearest,
                x="Ticker",
                y="Leverage",
                color="is_conviction",
                color_discrete_map={True: "#1a7431", False: "#4C6E91"},
                title=f"LEAPS Leverage Ratio — {opt_type} (Nearest Expiration)",
                labels={"Leverage": "Leverage (Spot / Premium)"},
                text="Leverage",
            )
            fig4.update_traces(texttemplate="%{text:.1f}x", textposition="outside")
            fig4.update_layout(showlegend=True)
            fig4.show()

## 6. Summary & Recommendation

Quick-reference table: best LEAPS pick per ticker at the ITM ~90% level, sorted by conviction then value.


In [71]:
if picks_df.empty:
    display(Markdown("❌ No picks to summarize."))
else:
    # Best ITM ~90% pick per ticker (nearest expiration)
    best = picks_df[picks_df["Type"] == "ITM ~90%"].copy()
    if best.empty:
        best = picks_df.copy()

    best = best.sort_values("DTE").groupby("Ticker").first().reset_index()

    # Sort: conviction first, then by breakeven %
    best["is_conviction"] = best["Ticker"].isin(CONVICTION)
    best = best.sort_values(["is_conviction", "Breakeven %"], ascending=[False, True])

    summary_cols = [
        "Ticker",
        "Conviction",
        "Expiration",
        "DTE",
        "Spot",
        "Strike",
        "Mid",
        "IV",
        "Breakeven",
        "Breakeven %",
        "Leverage",
        "Cost/Contract",
        "OI",
        "Spread %",
    ]
    available = [c for c in summary_cols if c in best.columns]

    display(Markdown("### 🎯 Top LEAPS Picks — One per Ticker"))
    display(
        best[available]
        .style.format(
            {
                "Spot": "${:,.2f}",
                "Strike": "${:,.0f}",
                "Mid": "${:,.2f}",
                "IV": "{:.1%}",
                "Breakeven": "${:,.2f}",
                "Breakeven %": "{:+.1%}",
                "Leverage": "{:.1f}x",
                "Cost/Contract": "${:,.0f}",
                "OI": "{:,.0f}",
                "Spread %": "{:.1%}",
            },
            na_rep="—",
        )
        .set_caption("ITM ~90% LEAPS Calls — Sorted by Conviction then Breakeven")
    )

    # Quick conviction summary
    conv_best = best[best["is_conviction"]].copy()
    if not conv_best.empty:
        display(Markdown("\n### ⭐ Your Conviction Picks at a Glance"))
        for _, row in conv_best.iterrows():
            ticker = row["Ticker"]
            display(
                Markdown(
                    f"**{ticker}** — ${row['Spot']:,.2f} spot → "
                    f"${row['Strike']:,.0f} strike LEAPS @ ${row['Mid']:,.2f} "
                    f"(${row['Cost/Contract']:,.0f}/contract) | "
                    f"Breakeven: ${row['Breakeven']:,.2f} ({row['Breakeven %']:+.1%}) | "
                    f"Leverage: {row['Leverage']:.1f}x | "
                    f"IV: {row['IV']:.1%} | "
                    f"Expires: {row['Expiration']} ({row['DTE']:.0f} DTE)"
                )
            )

### 🎯 Top LEAPS Picks — One per Ticker

,Ticker,Conviction,Expiration,DTE,Spot,Strike,Mid,IV,Breakeven,Breakeven %,Leverage,Cost/Contract,OI,Spread %
5,GEV,⭐,2026-05-15,96,$779.35,$700,$129.10,55.1%,$829.10,+6.4%,6.0x,"$12,910",135,4.3%
2,AVGO,⭐,2026-05-15,96,$332.92,$300,$56.55,58.4%,$356.55,+7.1%,5.9x,"$5,655",798,3.4%
0,AMAT,⭐,2026-05-15,96,$322.51,$290,$56.92,61.4%,$346.93,+7.6%,5.7x,"$5,692",333,4.1%
10,MSFT,,2026-05-15,96,$401.14,$360,$54.25,38.9%,$414.25,+3.3%,7.4x,"$5,425",92,3.9%
4,GE,,2026-05-15,96,$321.00,$290,$42.73,40.5%,$332.73,+3.7%,7.5x,"$4,272",610,5.3%
3,CMI,,2026-06-18,130,$577.73,$520,$79.85,35.6%,$599.85,+3.8%,7.2x,"$7,985",15,2.9%
9,META,,2026-05-15,96,$661.46,$600,$91.20,43.8%,$691.20,+4.5%,7.3x,"$9,120","12,442",3.2%
1,AMZN,,2026-05-15,96,$210.32,$190,$29.83,44.2%,$219.82,+4.5%,7.1x,"$2,983",233,1.8%
13,TSM,,2026-05-15,96,$348.85,$310,$54.70,48.0%,$364.70,+4.5%,6.4x,"$5,470","1,523",2.7%
6,ISRG,,2026-06-18,130,$488.15,$440,$70.70,41.0%,$510.70,+4.6%,6.9x,"$7,070",55,7.6%



### ⭐ Your Conviction Picks at a Glance

**GEV** — $779.35 spot → $700 strike LEAPS @ $129.10 ($12,910/contract) | Breakeven: $829.10 (+6.4%) | Leverage: 6.0x | IV: 55.1% | Expires: 2026-05-15 (96 DTE)

**AVGO** — $332.92 spot → $300 strike LEAPS @ $56.55 ($5,655/contract) | Breakeven: $356.55 (+7.1%) | Leverage: 5.9x | IV: 58.4% | Expires: 2026-05-15 (96 DTE)

**AMAT** — $322.51 spot → $290 strike LEAPS @ $56.92 ($5,692/contract) | Breakeven: $346.93 (+7.6%) | Leverage: 5.7x | IV: 61.4% | Expires: 2026-05-15 (96 DTE)

## 7. 🔍 Style Profile — Learning from Trades You Liked

These are trades you spotted and liked (but missed). Reverse-engineering the pattern to understand what you look for, then use it to find fresh setups.


In [72]:
# ── Trades you liked (missed) — use as style reference ──
ref_trades = pd.DataFrame(
    [
        {
            "ticker": "GEV",
            "strike": 450,
            "expiry": "2026-12-18",
            "dte": 316,
            "premium": 306.20,
        },
        {
            "ticker": "KLAC",
            "strike": 1320,
            "expiry": "2026-02-20",
            "dte": 14,
            "premium": 51.30,
        },
        {
            "ticker": "KLAC",
            "strike": 1340,
            "expiry": "2026-02-20",
            "dte": 14,
            "premium": 51.30,
        },
        {
            "ticker": "AVGO",
            "strike": 330,
            "expiry": "2026-09-18",
            "dte": 224,
            "premium": 52.70,
        },
        {
            "ticker": "AVGO",
            "strike": 330,
            "expiry": "2026-12-18",
            "dte": 316,
            "premium": 55.45,
        },
        {
            "ticker": "WDC",
            "strike": 240,
            "expiry": "2026-02-20",
            "dte": 14,
            "premium": 22.50,
        },
    ]
)

# Compute style metrics for each reference trade
style_rows = []
for _, t in ref_trades.iterrows():
    spot = spot_map.get(t["ticker"])
    if not spot:
        continue
    moneyness = t["strike"] / spot
    intrinsic = max(spot - t["strike"], 0)
    extrinsic = t["premium"] - intrinsic
    extrinsic_pct = extrinsic / t["premium"] if t["premium"] > 0 else 0
    breakeven = t["strike"] + t["premium"]
    breakeven_pct = (breakeven - spot) / spot
    leverage = spot / t["premium"] if t["premium"] > 0 else 0
    value_at_risk = t["premium"] * 100

    style_rows.append(
        {
            "Ticker": t["ticker"],
            "Strike": t["strike"],
            "Expiry": t["expiry"],
            "DTE": t["dte"],
            "Spot": spot,
            "Premium": t["premium"],
            "Moneyness": moneyness,
            "Intrinsic": intrinsic,
            "Extrinsic": extrinsic,
            "Extrinsic %": extrinsic_pct,
            "Breakeven": breakeven,
            "Breakeven %": breakeven_pct,
            "Leverage": leverage,
            "Value at Risk": value_at_risk,
            "Category": "LEAPS" if t["dte"] > 90 else "Swing",
        }
    )

style_df = pd.DataFrame(style_rows)

display(Markdown("### 🧬 Your Style DNA — What You Like"))
display(
    style_df.style.format(
        {
            "Strike": "${:,.0f}",
            "Spot": "${:,.2f}",
            "Premium": "${:,.2f}",
            "Moneyness": "{:.1%}",
            "Intrinsic": "${:,.2f}",
            "Extrinsic": "${:,.2f}",
            "Extrinsic %": "{:.1%}",
            "Breakeven": "${:,.2f}",
            "Breakeven %": "{:+.1%}",
            "Leverage": "{:.1f}x",
            "Value at Risk": "${:,.0f}",
        },
        na_rep="—",
    ).set_caption("Reference trades — what caught your eye")
)

# Extract the pattern
leaps_trades = style_df[style_df["Category"] == "LEAPS"]
swing_trades = style_df[style_df["Category"] == "Swing"]

display(Markdown("\n### 📊 Pattern Summary"))

if not leaps_trades.empty:
    display(
        Markdown(
            f"**LEAPS pattern** (DTE > 90):\n"
            f"- Moneyness: {leaps_trades['Moneyness'].mean():.0%} avg "
            f"(range {leaps_trades['Moneyness'].min():.0%}–{leaps_trades['Moneyness'].max():.0%})\n"
            f"- Breakeven above spot: {leaps_trades['Breakeven %'].mean():+.1%} avg\n"
            f"- Leverage: {leaps_trades['Leverage'].mean():.1f}x avg\n"
            f"- Extrinsic %: {leaps_trades['Extrinsic %'].mean():.0%} avg (lower = more intrinsic value)\n"
            f"- Avg value at risk: ${leaps_trades['Value at Risk'].mean():,.0f}/contract\n"
            f"- **Style: Deep-to-moderate ITM, long-dated, stock replacement with leverage**"
        )
    )

if not swing_trades.empty:
    display(
        Markdown(
            f"\n**Swing pattern** (DTE ≤ 90):\n"
            f"- Moneyness: {swing_trades['Moneyness'].mean():.0%} avg\n"
            f"- Breakeven above spot: {swing_trades['Breakeven %'].mean():+.1%} avg\n"
            f"- Leverage: {swing_trades['Leverage'].mean():.1f}x avg\n"
            f"- Avg value at risk: ${swing_trades['Value at Risk'].mean():,.0f}/contract\n"
            f"- **Style: Near-ATM, short-duration momentum plays with high leverage**"
        )
    )

# Store style parameters for the scoring engine
STYLE_PREFS = {
    "preferred_moneyness_leaps": (0.55, 1.00),  # deep ITM to ATM
    "preferred_moneyness_swing": (0.85, 1.00),  # near ATM
    "ideal_moneyness_leaps": float(leaps_trades["Moneyness"].mean())
    if not leaps_trades.empty
    else 0.90,
    "ideal_breakeven_pct": float(style_df["Breakeven %"].mean()),
    "max_var_per_contract": float(style_df["Value at Risk"].max()),
    "ideal_leverage": float(style_df["Leverage"].mean()),
}
display(Markdown(f"\n*Style prefs saved → feeding into scoring engine below*"))

### 🧬 Your Style DNA — What You Like

,Ticker,Strike,Expiry,DTE,Spot,Premium,Moneyness,Intrinsic,Extrinsic,Extrinsic %,Breakeven,Breakeven %,Leverage,Value at Risk,Category
0,GEV,$450,2026-12-18,316,$779.35,$306.20,57.7%,$329.35,$-23.15,-7.6%,$756.20,-3.0%,2.5x,"$30,620",LEAPS
1,KLAC,"$1,320",2026-02-20,14,"$1,442.95",$51.30,91.5%,$122.95,$-71.65,-139.7%,"$1,371.30",-5.0%,28.1x,"$5,130",Swing
2,KLAC,"$1,340",2026-02-20,14,"$1,442.95",$51.30,92.9%,$102.95,$-51.65,-100.7%,"$1,391.30",-3.6%,28.1x,"$5,130",Swing
3,AVGO,$330,2026-09-18,224,$332.92,$52.70,99.1%,$2.92,$49.78,94.5%,$382.70,+15.0%,6.3x,"$5,270",LEAPS
4,AVGO,$330,2026-12-18,316,$332.92,$55.45,99.1%,$2.92,$52.53,94.7%,$385.45,+15.8%,6.0x,"$5,545",LEAPS
5,WDC,$240,2026-02-20,14,$282.58,$22.50,84.9%,$42.58,$-20.08,-89.2%,$262.50,-7.1%,12.6x,"$2,250",Swing



### 📊 Pattern Summary

**LEAPS pattern** (DTE > 90):
- Moneyness: 85% avg (range 58%–99%)
- Breakeven above spot: +9.3% avg
- Leverage: 5.0x avg
- Extrinsic %: 61% avg (lower = more intrinsic value)
- Avg value at risk: $13,812/contract
- **Style: Deep-to-moderate ITM, long-dated, stock replacement with leverage**


**Swing pattern** (DTE ≤ 90):
- Moneyness: 90% avg
- Breakeven above spot: -5.2% avg
- Leverage: 22.9x avg
- Avg value at risk: $4,170/contract
- **Style: Near-ATM, short-duration momentum plays with high leverage**


*Style prefs saved → feeding into scoring engine below*

## 8. 🧠 Edge Scoring Engine — Find the Next Plays

Using your style profile + market data to score **every LEAPS option** on edge & probability of profit.

| Factor              | What it measures                                     | Weight |
| ------------------- | ---------------------------------------------------- | ------ |
| **P(Profit) — BSM** | Black-Scholes probability spot > breakeven at expiry | 25%    |
| **IV vs HV edge**   | IV overpriced vs realized? Trend-adjusted            | 15%    |
| **Breakeven %**     | How far stock must move — lower = better             | 15%    |
| **Momentum**        | 1M + 3M returns — trend is your friend               | 20%    |
| **Style match**     | How close to your preferred moneyness/leverage       | 15%    |
| **Liquidity**       | Tight spreads + open interest                        | 10%    |


In [83]:
from scipy.stats import norm

# ══════════════════════════════════════════════════════════════════
# STEP 1: Fetch HV + Momentum for all tickers
# ══════════════════════════════════════════════════════════════════
hv_momentum = {}

for ticker in ALL_TICKERS:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period="1y")
        if hist.empty or "Close" not in hist.columns:
            continue
        closes = hist["Close"].dropna()
        if len(closes) < 60:
            continue

        log_ret = np.log(closes / closes.shift(1)).dropna()
        hv_30 = float(log_ret.iloc[-30:].std() * math.sqrt(252))
        hv_60 = float(log_ret.iloc[-60:].std() * math.sqrt(252))

        price = float(closes.iloc[-1])
        ret_1m = (
            float(closes.iloc[-1] / closes.iloc[-21] - 1) if len(closes) > 21 else 0
        )
        ret_3m = (
            float(closes.iloc[-1] / closes.iloc[-63] - 1) if len(closes) > 63 else 0
        )
        ret_6m = (
            float(closes.iloc[-1] / closes.iloc[-126] - 1) if len(closes) > 126 else 0
        )

        # RSI (Wilder's smoothing — EWM with com=13)
        delta = closes.diff()
        gain = delta.clip(lower=0).ewm(com=13, adjust=False).mean()
        loss_s = (-delta.clip(upper=0)).ewm(com=13, adjust=False).mean()
        rs = gain.iloc[-1] / loss_s.iloc[-1] if loss_s.iloc[-1] > 0 else 100
        rsi = float(100 - (100 / (1 + rs)))

        # 50/200 MA trend
        ma50 = float(closes.rolling(50).mean().iloc[-1]) if len(closes) >= 50 else None
        ma200 = (
            float(closes.rolling(200).mean().iloc[-1]) if len(closes) >= 200 else None
        )
        above_50 = price > ma50 if ma50 else False
        above_200 = price > ma200 if ma200 else False
        golden_cross = (ma50 > ma200) if (ma50 and ma200) else False

        # 52-week high distance
        high_52w = (
            float(closes.iloc[-252:].max())
            if len(closes) >= 252
            else float(closes.max())
        )
        dist_52w = (price - high_52w) / high_52w

        hv_momentum[ticker] = {
            "hv_30": hv_30,
            "hv_60": hv_60,
            "ret_1m": ret_1m,
            "ret_3m": ret_3m,
            "ret_6m": ret_6m,
            "rsi": rsi,
            "dist_52w_high": dist_52w,
            "above_50ma": above_50,
            "above_200ma": above_200,
            "golden_cross": golden_cross,
        }
        trend_flags = []
        if above_50:
            trend_flags.append("▲50MA")
        if above_200:
            trend_flags.append("▲200MA")
        if golden_cross:
            trend_flags.append("🔥GC")
        print(
            f"  {ticker}: HV30={hv_30:.1%} 1M={ret_1m:+.1%} 3M={ret_3m:+.1%} RSI={rsi:.0f} {' '.join(trend_flags)}"
        )
    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")

print(f"\n✅ HV/momentum for {len(hv_momentum)} tickers")

  WDC: HV30=94.8% 1M=+50.6% 3M=+72.9% RSI=64 ▲50MA ▲200MA 🔥GC
  GEV: HV30=42.7% 1M=+24.0% 3M=+41.8% RSI=68 ▲50MA ▲200MA 🔥GC
  STX: HV30=88.9% 1M=+50.9% 3M=+54.6% RSI=65 ▲50MA ▲200MA 🔥GC
  LRCX: HV30=66.1% 1M=+15.0% 3M=+42.7% RSI=58 ▲50MA ▲200MA 🔥GC
  AMAT: HV30=53.4% 1M=+14.5% 3M=+38.4% RSI=57 ▲50MA ▲200MA 🔥GC
  TSM: HV30=36.5% 1M=+9.7% 3M=+20.9% RSI=62 ▲50MA ▲200MA 🔥GC
  GE: HV30=37.8% 1M=+2.1% 3M=+5.3% RSI=59 ▲50MA ▲200MA 🔥GC
  CMI: HV30=44.6% 1M=+6.1% 3M=+25.4% RSI=54 ▲50MA ▲200MA 🔥GC
  KLAC: HV30=73.7% 1M=+8.9% 3M=+19.8% RSI=52 ▲50MA ▲200MA 🔥GC
  SNPS: HV30=43.8% 1M=-17.0% 3M=+7.8% RSI=34 
  META: HV30=41.4% 1M=+2.4% 3M=+7.0% RSI=49 ▲50MA
  UBER: HV30=32.2% 1M=-14.6% 3M=-18.8% RSI=32 
  ISRG: HV30=24.6% 1M=-16.6% 3M=-10.9% RSI=29 🔥GC
  MSFT: HV30=39.9% 1M=-16.1% 3M=-19.2% RSI=29 
  AMZN: HV30=32.9% 1M=-14.6% 3M=-13.5% RSI=29 🔥GC
  AVGO: HV30=39.3% 1M=+0.1% 3M=-6.2% RSI=49 ▲200MA 🔥GC

✅ HV/momentum for 16 tickers


### Scoring & Ranking — LEAPS + Swing Opportunities


In [84]:
# ══════════════════════════════════════════════════════════════════
# STEP 2: Score every LEAPS option
# ══════════════════════════════════════════════════════════════════


def bsm_prob_profit(spot, breakeven, iv, dte, r=None):
    """P(spot > breakeven at expiry) under Black-Scholes lognormal model.

    NOTE: Uses risk-neutral drift (r), so this is the risk-neutral probability.
    Real-world probability would use expected return μ > r (equity risk premium),
    making this estimate conservative — actual P(profit) is likely higher.
    """
    if r is None:
        r = RISK_FREE_RATE
    if iv <= 0 or dte <= 0 or spot <= 0 or breakeven <= 0:
        return np.nan
    T = dte / 365.0
    d2 = (math.log(spot / breakeven) + (r - 0.5 * iv**2) * T) / (iv * math.sqrt(T))
    return float(norm.cdf(d2))


def score_option(row, hv_data, style_prefs):
    """Score a single option 0-100 based on edge factors."""
    scores = {}
    weights = {}
    ticker = row["ticker"]
    hv = hv_data.get(ticker, {})
    if not hv:
        return np.nan, np.nan, {}

    spot = row["spot"]
    iv = row.get("iv", np.nan)
    dte = row["dte"]
    breakeven = row["breakeven"]
    moneyness = row["moneyness"]

    # 1. P(Profit) from BSM — 25%
    p_profit = bsm_prob_profit(spot, breakeven, iv, dte) if not np.isnan(iv) else np.nan
    if not np.isnan(p_profit):
        scores["p_profit"] = min(p_profit * 100, 100)  # 0-100
        weights["p_profit"] = 0.25

    # 2. Momentum — 20% (combine 1M + 3M, RSI, trend flags)
    ret_1m = hv.get("ret_1m", 0)
    ret_3m = hv.get("ret_3m", 0)
    rsi = hv.get("rsi", 50)
    above_50 = hv.get("above_50ma", False)
    above_200 = hv.get("above_200ma", False)
    golden_cross = hv.get("golden_cross", False)

    # Momentum: positive returns = good, RSI 50-70 sweet spot (trending but not overbought)
    mom_score = 0
    mom_score += np.clip(ret_1m * 200, -30, 40)  # 1M return contribution
    mom_score += np.clip(ret_3m * 100, -20, 30)  # 3M return contribution
    if 45 < rsi < 72:
        mom_score += 15  # sweet spot — trending, not overbought
    elif rsi >= 72:
        mom_score += 5  # overbought risk, but still going
    if above_50:
        mom_score += 5
    if above_200:
        mom_score += 5
    if golden_cross:
        mom_score += 5
    scores["momentum"] = np.clip(mom_score, 0, 100)
    weights["momentum"] = 0.20

    # 3. Breakeven % — 15% (lower = easier to profit)
    be_pct = row.get("breakeven_pct", np.nan)
    if not np.isnan(be_pct):
        # Best case: breakeven below spot (already ITM beyond premium)
        # Worst: breakeven +20% above
        be_score = np.clip((0.15 - be_pct) / 0.20 * 100, 0, 100)
        scores["breakeven"] = be_score
        weights["breakeven"] = 0.15

    # 4. IV vs HV edge — 15%
    hv_30 = hv.get("hv_30", np.nan)
    if not np.isnan(iv) and not np.isnan(hv_30) and iv > 0:
        # For CALL buyers with momentum: HV > IV means realized moves exceed what IV prices in
        # Also reward: IV not wildly expensive vs HV (IV/HV < 1.3 is good)
        iv_hv_ratio = iv / hv_30 if hv_30 > 0 else 2.0
        if iv_hv_ratio < 0.9:
            iv_score = 90  # IV cheap vs realized — strong edge
        elif iv_hv_ratio < 1.1:
            iv_score = 70  # fair value
        elif iv_hv_ratio < 1.3:
            iv_score = 45  # slightly expensive
        else:
            iv_score = max(0, 30 - (iv_hv_ratio - 1.3) * 50)  # expensive
        scores["iv_edge"] = iv_score
        weights["iv_edge"] = 0.15

    # 5. Style match — 15% (how close to your pattern)
    ideal_mon = style_prefs.get("ideal_moneyness_leaps", 0.90)
    mon_dist = abs(moneyness - ideal_mon)
    style_score = max(0, 100 - mon_dist * 500)  # penalize distance from ideal
    # Bonus for deep ITM (more intrinsic, less risk)
    if moneyness < 0.90:
        style_score = min(100, style_score + 10)
    scores["style_match"] = style_score
    weights["style_match"] = 0.15

    # 6. Liquidity — 10%
    oi = safe_float(row.get("openInterest", 0))
    spread_pct = row.get("spread_pct", np.nan)
    liq_score = 0
    if oi > 500:
        liq_score += 40
    elif oi > 100:
        liq_score += 25
    elif oi > 20:
        liq_score += 10
    if not np.isnan(spread_pct):
        if spread_pct < 0.05:
            liq_score += 60
        elif spread_pct < 0.10:
            liq_score += 40
        elif spread_pct < 0.20:
            liq_score += 20
    scores["liquidity"] = min(liq_score, 100)
    weights["liquidity"] = 0.10

    # Weighted total
    total_w = sum(weights.values())
    if total_w == 0:
        return np.nan, p_profit, scores

    edge_score = sum(scores[k] * weights[k] for k in scores) / total_w
    return edge_score, p_profit, scores


# ── Score all LEAPS options ──
scored_rows = []

for _, row in leaps_df.iterrows():
    edge, p_profit, components = score_option(row, hv_momentum, STYLE_PREFS)
    if np.isnan(edge):
        continue

    scored_rows.append(
        {
            "Ticker": row["ticker"],
            "Conviction": "⭐" if row["ticker"] in CONVICTION else "",
            "Expiry": row["expiration"],
            "DTE": row["dte"],
            "Strike": row["strike"],
            "Moneyness": row["moneyness"],
            "Spot": row["spot"],
            "Mid": row["mid"],
            "IV": row.get("iv", np.nan),
            "HV30": hv_momentum.get(row["ticker"], {}).get("hv_30", np.nan),
            "Breakeven": row["breakeven"],
            "Breakeven %": row["breakeven_pct"],
            "Leverage": row["leverage"],
            "Cost/Contract": row["cost_per_contract"],
            "OI": safe_float(row.get("openInterest", 0)),
            "Spread %": row.get("spread_pct", np.nan),
            "P(Profit)": p_profit,
            "Edge Score": edge,
            "s_pprofit": components.get("p_profit", np.nan),
            "s_momentum": components.get("momentum", np.nan),
            "s_breakeven": components.get("breakeven", np.nan),
            "s_iv_edge": components.get("iv_edge", np.nan),
            "s_style": components.get("style_match", np.nan),
            "s_liquidity": components.get("liquidity", np.nan),
            "Ret 1M": hv_momentum.get(row["ticker"], {}).get("ret_1m", np.nan),
            "Ret 3M": hv_momentum.get(row["ticker"], {}).get("ret_3m", np.nan),
            "RSI": hv_momentum.get(row["ticker"], {}).get("rsi", np.nan),
        }
    )

scored_df = pd.DataFrame(scored_rows)
print(
    f"✅ Scored {len(scored_df)} options across {scored_df['Ticker'].nunique()} tickers"
)

✅ Scored 536 options across 16 tickers


## 9. 🏆 Top Ranked Opportunities — Best Edge Right Now


In [75]:
if scored_df.empty:
    display(Markdown("❌ No scored options."))
else:
    # ── Best LEAPS pick per ticker (highest edge score, prefer ITM ~85-95%) ──
    sweet_spot = scored_df[
        (scored_df["Moneyness"] >= 0.82) & (scored_df["Moneyness"] <= 0.98)
    ].copy()
    if sweet_spot.empty:
        sweet_spot = scored_df.copy()

    # Best single option per ticker (highest edge)
    top_per_ticker = (
        sweet_spot.sort_values("Edge Score", ascending=False)
        .groupby("Ticker")
        .first()
        .reset_index()
    )
    top_per_ticker = top_per_ticker.sort_values("Edge Score", ascending=False)

    display_cols = [
        "Ticker",
        "Conviction",
        "Expiry",
        "DTE",
        "Strike",
        "Moneyness",
        "Spot",
        "Mid",
        "IV",
        "HV30",
        "Breakeven",
        "Breakeven %",
        "Leverage",
        "Cost/Contract",
        "P(Profit)",
        "Edge Score",
        "Ret 1M",
        "Ret 3M",
        "RSI",
        "OI",
    ]
    avail = [c for c in display_cols if c in top_per_ticker.columns]

    display(Markdown("### 🎯 Best LEAPS Pick per Ticker — Ranked by Edge Score"))
    display(
        top_per_ticker[avail]
        .style.format(
            {
                "Strike": "${:,.0f}",
                "Spot": "${:,.2f}",
                "Mid": "${:,.2f}",
                "Moneyness": "{:.0%}",
                "IV": "{:.1%}",
                "HV30": "{:.1%}",
                "Breakeven": "${:,.2f}",
                "Breakeven %": "{:+.1%}",
                "Leverage": "{:.1f}x",
                "Cost/Contract": "${:,.0f}",
                "P(Profit)": "{:.0%}",
                "Edge Score": "{:.0f}",
                "Ret 1M": "{:+.1%}",
                "Ret 3M": "{:+.1%}",
                "RSI": "{:.0f}",
                "OI": "{:,.0f}",
            },
            na_rep="—",
        )
        .background_gradient(subset=["Edge Score"], cmap="RdYlGn", vmin=30, vmax=80)
        .background_gradient(subset=["P(Profit)"], cmap="RdYlGn", vmin=0.3, vmax=0.8)
        .set_caption("Ranked by Edge Score — higher = stronger setup")
    )

    # ── Score component breakdown for top 8 ──
    top8 = top_per_ticker.head(8)
    component_cols = [
        "Ticker",
        "Edge Score",
        "s_pprofit",
        "s_momentum",
        "s_breakeven",
        "s_iv_edge",
        "s_style",
        "s_liquidity",
    ]
    comp_avail = [c for c in component_cols if c in top8.columns]
    display(Markdown("\n### 🔬 Score Breakdown — Top 8"))
    display(
        top8[comp_avail]
        .style.format(
            {
                "Edge Score": "{:.0f}",
                "s_pprofit": "{:.0f}",
                "s_momentum": "{:.0f}",
                "s_breakeven": "{:.0f}",
                "s_iv_edge": "{:.0f}",
                "s_style": "{:.0f}",
                "s_liquidity": "{:.0f}",
            },
            na_rep="—",
        )
        .background_gradient(
            cmap="RdYlGn",
            vmin=20,
            vmax=90,
            axis=None,
            subset=[
                "s_pprofit",
                "s_momentum",
                "s_breakeven",
                "s_iv_edge",
                "s_style",
                "s_liquidity",
            ],
        )
        .set_caption(
            "Component scores (0-100) | P(Profit)=25% Momentum=20% Breakeven=15% IV Edge=15% Style=15% Liquidity=10%"
        )
    )

    # ── Chart: Edge score vs P(Profit) scatter ──
    plot_df = top_per_ticker.copy()
    plot_df["is_conviction"] = plot_df["Ticker"].isin(CONVICTION)

    fig = px.scatter(
        plot_df,
        x="P(Profit)",
        y="Edge Score",
        color="is_conviction",
        text="Ticker",
        size="Leverage",
        size_max=25,
        color_discrete_map={True: "#1a7431", False: "#4C6E91"},
        title="Edge Score vs P(Profit) — Best LEAPS per Ticker",
        labels={"is_conviction": "Conviction"},
    )
    fig.update_traces(textposition="top center")
    fig.update_layout(
        xaxis_tickformat=".0%",
        xaxis_title="Probability of Profit (BSM)",
        yaxis_title="Edge Score (0-100)",
    )
    # Quadrant lines
    fig.add_hline(y=55, line_dash="dot", line_color="gray", opacity=0.5)
    fig.add_vline(x=0.50, line_dash="dot", line_color="gray", opacity=0.5)
    fig.add_annotation(
        x=0.7,
        y=75,
        text="🎯 Sweet Spot",
        showarrow=False,
        font=dict(size=14, color="green"),
    )
    fig.show()

### 🎯 Best LEAPS Pick per Ticker — Ranked by Edge Score

,Ticker,Conviction,Expiry,DTE,Strike,Moneyness,Spot,Mid,IV,HV30,Breakeven,Breakeven %,Leverage,Cost/Contract,P(Profit),Edge Score,Ret 1M,Ret 3M,RSI,OI
3,CMI,,2026-06-18,130,$480,83%,$577.73,$110.75,38.6%,44.6%,$590.75,+2.3%,5.2x,"$11,075",44%,69,+6.1%,+25.4%,50,38
12,STX,,2026-06-18,130,$370,86%,$429.32,$108.65,79.7%,88.9%,$478.65,+11.5%,4.0x,"$10,865",33%,68,+50.9%,+54.6%,70,132
8,LRCX,,2026-06-18,130,$190,82%,$231.01,$59.73,71.4%,66.1%,$249.72,+8.1%,3.9x,"$5,972",36%,67,+15.0%,+42.7%,54,530
5,GEV,⭐,2026-05-15,96,$690,89%,$779.35,$135.45,55.4%,42.7%,$825.45,+5.9%,5.8x,"$13,545",38%,66,+24.0%,+41.8%,71,117
0,AMAT,⭐,2026-06-18,130,$270,84%,$322.51,$73.75,60.4%,53.4%,$343.75,+6.6%,4.4x,"$7,375",38%,65,+14.5%,+38.4%,48,625
15,WDC,,2026-06-18,130,$250,88%,$282.58,$71.42,85.2%,94.8%,$321.43,+13.7%,4.0x,"$7,142",32%,64,+50.6%,+72.9%,66,545
7,KLAC,,2026-06-18,130,"$1,200",83%,"$1,442.95",$325.40,56.8%,73.7%,"$1,525.40",+5.7%,4.4x,"$32,540",39%,64,+8.9%,+19.8%,43,107
13,TSM,,2026-05-15,96,$300,86%,$348.85,$62.00,48.7%,36.5%,$362.00,+3.8%,5.6x,"$6,200",41%,62,+9.7%,+20.9%,53,"42,582"
9,META,,2026-05-15,96,$580,88%,$661.46,$105.85,45.2%,41.4%,$685.85,+3.7%,6.2x,"$10,585",41%,59,+2.4%,+7.0%,59,226
4,GE,,2026-06-18,130,$290,90%,$321.00,$45.83,39.0%,37.8%,$335.82,+4.6%,7.0x,"$4,582",40%,57,+2.1%,+5.3%,48,618



### 🔬 Score Breakdown — Top 8

,Ticker,Edge Score,s_pprofit,s_momentum,s_breakeven,s_iv_edge,s_style,s_liquidity
3,CMI,69,44,68,64,90,99,70
12,STX,68,33,100,18,90,100,85
8,LRCX,67,36,90,34,70,95,100
5,GEV,66,38,100,45,45,94,85
0,AMAT,65,38,89,42,45,100,100
15,WDC,64,32,100,6,90,94,80
7,KLAC,64,39,53,46,90,99,85
13,TSM,62,41,70,56,28,100,100


## 10. ⚡ Short-Term Momentum Scanner — KLAC-Style Swing Plays

Your KLAC trades were 14-DTE near-ATM calls with massive leverage (22x).  
Scanning for similar setups: **strong momentum + near-ATM + 14-45 DTE** across all tickers.


In [76]:
# ══════════════════════════════════════════════════════════════════
# Scan for short-term momentum plays (14-45 DTE, near ATM)
# Like your KLAC $1320/$1340 calls — high leverage swing trades
# ══════════════════════════════════════════════════════════════════

SWING_DTE_MIN = 7
SWING_DTE_MAX = 50
SWING_MONEYNESS = (0.93, 1.03)  # near ATM

swing_rows = []
swing_skipped = []

# Filter tickers by momentum first — only scan tickers with positive trend
momentum_tickers = []
for ticker in ALL_TICKERS:
    hv = hv_momentum.get(ticker, {})
    ret_1m = hv.get("ret_1m", 0)
    ret_3m = hv.get("ret_3m", 0)
    rsi = hv.get("rsi", 50)
    above_50 = hv.get("above_50ma", False)

    # Momentum filter: at least 2 of these
    checks = [ret_1m > 0.02, ret_3m > 0.05, rsi > 50, above_50]
    if sum(checks) >= 2:
        momentum_tickers.append(ticker)

print(f"Momentum tickers ({len(momentum_tickers)}): {', '.join(momentum_tickers)}\n")

for ticker in momentum_tickers:
    spot = spot_map.get(ticker)
    if not spot:
        continue

    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        all_exps = t.options or []

        today = datetime.now().date()
        swing_exps = []
        for exp_str in all_exps:
            try:
                exp_date = datetime.strptime(exp_str, "%Y-%m-%d").date()
                dte = (exp_date - today).days
                if SWING_DTE_MIN <= dte <= SWING_DTE_MAX:
                    swing_exps.append((exp_str, dte))
            except ValueError:
                continue

        if not swing_exps:
            swing_skipped.append((ticker, "no_swing_exps"))
            continue

        # Take nearest 2 expirations
        swing_exps.sort(key=lambda x: x[1])
        for exp_str, dte in swing_exps[:2]:
            time.sleep(RATE_LIMIT_SLEEP)
            chain = t.option_chain(exp_str)
            calls = chain.calls.copy()
            if calls.empty:
                continue

            calls["mid"] = (calls["bid"] + calls["ask"]) / 2
            calls.loc[calls["mid"] <= 0, "mid"] = calls["lastPrice"]
            calls["moneyness"] = calls["strike"] / spot
            calls["spread_pct"] = np.where(
                calls["mid"] > 0, (calls["ask"] - calls["bid"]) / calls["mid"], np.nan
            )

            # Filter to near ATM
            mask = (
                (calls["moneyness"] >= SWING_MONEYNESS[0])
                & (calls["moneyness"] <= SWING_MONEYNESS[1])
                & (calls["mid"] > 0.5)
            )
            near_atm = calls[mask].copy()
            if near_atm.empty:
                continue

            for _, opt in near_atm.iterrows():
                iv = safe_float(opt.get("impliedVolatility"))
                intrinsic = max(spot - opt["strike"], 0)
                extrinsic = opt["mid"] - intrinsic
                breakeven = opt["strike"] + opt["mid"]
                breakeven_pct = (breakeven - spot) / spot
                leverage = spot / opt["mid"] if opt["mid"] > 0 else 0

                p_profit = bsm_prob_profit(spot, breakeven, iv, dte)
                hv = hv_momentum.get(ticker, {})

                # Quick edge score for swing
                edge = 0
                n = 0
                if not np.isnan(p_profit):
                    edge += p_profit * 100 * 0.20
                    n += 0.20
                # Momentum (big weight for swings)
                ret_1m = hv.get("ret_1m", 0)
                ret_3m = hv.get("ret_3m", 0)
                mom = np.clip(ret_1m * 300, -30, 50) + np.clip(ret_3m * 150, -20, 30)
                if hv.get("above_50ma"):
                    mom += 5
                if hv.get("golden_cross"):
                    mom += 5
                edge += np.clip(mom, 0, 100) * 0.35
                n += 0.35
                # Breakeven closeness
                be_score = np.clip((0.08 - breakeven_pct) / 0.12 * 100, 0, 100)
                edge += be_score * 0.20
                n += 0.20
                # IV edge
                hv30 = hv.get("hv_30", np.nan)
                if not np.isnan(iv) and not np.isnan(hv30) and hv30 > 0:
                    ratio = iv / hv30
                    iv_s = 80 if ratio < 1.0 else max(0, 70 - (ratio - 1.0) * 80)
                    edge += iv_s * 0.15
                    n += 0.15
                # Liquidity
                oi = safe_float(opt.get("openInterest", 0))
                sp = opt["spread_pct"] if not np.isnan(opt["spread_pct"]) else 0.5
                liq = min(
                    100,
                    (30 if oi > 100 else 10)
                    + (60 if sp < 0.10 else 20 if sp < 0.20 else 0),
                )
                edge += liq * 0.10
                n += 0.10

                edge = edge / n if n > 0 else 0

                swing_rows.append(
                    {
                        "Ticker": ticker,
                        "Conviction": "⭐" if ticker in CONVICTION else "",
                        "Expiry": exp_str,
                        "DTE": dte,
                        "Strike": opt["strike"],
                        "Moneyness": opt["moneyness"],
                        "Spot": spot,
                        "Mid": opt["mid"],
                        "IV": iv,
                        "Breakeven": breakeven,
                        "Breakeven %": breakeven_pct,
                        "Leverage": leverage,
                        "Cost/Contract": opt["mid"] * 100,
                        "P(Profit)": p_profit,
                        "Edge Score": edge,
                        "OI": oi,
                        "Spread %": opt["spread_pct"],
                        "Ret 1M": hv.get("ret_1m", np.nan),
                        "Ret 3M": hv.get("ret_3m", np.nan),
                        "RSI": hv.get("rsi", np.nan),
                    }
                )

        print(
            f"  ✅ {ticker} — {len([r for r in swing_rows if r['Ticker'] == ticker])} swing options found"
        )
    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")

swing_df = pd.DataFrame(swing_rows)
print(f"\n✅ Total swing candidates: {len(swing_df)}")

# ── Show top swing picks ──
if not swing_df.empty:
    # Best per ticker
    swing_top = (
        swing_df.sort_values("Edge Score", ascending=False)
        .groupby("Ticker")
        .head(2)
        .reset_index(drop=True)
    )
    swing_top = swing_top.sort_values("Edge Score", ascending=False).head(20)

    display(Markdown("### ⚡ Top Swing Plays — KLAC-Style Momentum Calls"))
    display(
        Markdown(
            "*Short DTE + near ATM + strong momentum = high leverage plays like your KLAC $1320/$1340 trades*"
        )
    )

    scols = [
        "Ticker",
        "Conviction",
        "Expiry",
        "DTE",
        "Strike",
        "Moneyness",
        "Spot",
        "Mid",
        "IV",
        "Breakeven",
        "Breakeven %",
        "Leverage",
        "Cost/Contract",
        "P(Profit)",
        "Edge Score",
        "Ret 1M",
        "Ret 3M",
        "RSI",
        "OI",
    ]
    sa = [c for c in scols if c in swing_top.columns]

    display(
        swing_top[sa]
        .style.format(
            {
                "Strike": "${:,.0f}",
                "Spot": "${:,.2f}",
                "Mid": "${:,.2f}",
                "Moneyness": "{:.0%}",
                "IV": "{:.1%}",
                "Breakeven": "${:,.2f}",
                "Breakeven %": "{:+.1%}",
                "Leverage": "{:.0f}x",
                "Cost/Contract": "${:,.0f}",
                "P(Profit)": "{:.0%}",
                "Edge Score": "{:.0f}",
                "Ret 1M": "{:+.1%}",
                "Ret 3M": "{:+.1%}",
                "RSI": "{:.0f}",
                "OI": "{:,.0f}",
                "Spread %": "{:.1%}",
            },
            na_rep="—",
        )
        .background_gradient(subset=["Edge Score"], cmap="RdYlGn", vmin=30, vmax=75)
        .background_gradient(subset=["P(Profit)"], cmap="RdYlGn", vmin=0.2, vmax=0.6)
        .set_caption("Swing Calls — Ranked by Edge Score")
    )

    # Scenario: what if stock moves +5%, +10% before expiry?
    display(Markdown("\n### 💰 Quick Scenario — If Stock Pops"))
    best_swings = swing_top.groupby("Ticker").first().reset_index().head(8)
    scen_rows = []
    for _, r in best_swings.iterrows():
        for pct_move in [0.05, 0.10, 0.15]:
            new_spot = r["Spot"] * (1 + pct_move)
            payout = max(new_spot - r["Strike"], 0)
            gain = payout - r["Mid"]
            gain_pct = gain / r["Mid"] if r["Mid"] > 0 else 0
            scen_rows.append(
                {
                    "Ticker": r["Ticker"],
                    "Strike": r["Strike"],
                    "DTE": r["DTE"],
                    "Cost": r["Mid"],
                    "Stock Move": pct_move,
                    "New Spot": new_spot,
                    "Payout": payout,
                    "Gain/Loss": gain,
                    "Return %": gain_pct,
                }
            )
    scen_df = pd.DataFrame(scen_rows)
    pivot = scen_df.pivot_table(
        index=["Ticker", "Strike", "DTE", "Cost"],
        columns="Stock Move",
        values="Return %",
    )
    pivot.columns = [f"+{c:.0%}" for c in pivot.columns]
    display(
        pivot.style.format("{:+.0%}", na_rep="—")
        .background_gradient(cmap="RdYlGn", vmin=-0.5, vmax=3, axis=None)
        .set_caption("Return % if stock moves +5% / +10% / +15% before expiry")
    )
else:
    display(Markdown("No swing setups found meeting criteria."))

Momentum tickers (10): WDC, GEV, STX, LRCX, AMAT, TSM, GE, CMI, KLAC, META

  ✅ WDC — 15 swing options found
  ✅ GEV — 27 swing options found
  ✅ STX — 14 swing options found
  ✅ LRCX — 15 swing options found
  ✅ AMAT — 20 swing options found
  ✅ TSM — 21 swing options found
  ✅ GE — 20 swing options found
  ✅ CMI — 12 swing options found
  ✅ KLAC — 21 swing options found
  ✅ META — 33 swing options found

✅ Total swing candidates: 198


### ⚡ Top Swing Plays — KLAC-Style Momentum Calls

*Short DTE + near ATM + strong momentum = high leverage plays like your KLAC $1320/$1340 trades*

,Ticker,Conviction,Expiry,DTE,Strike,Moneyness,Spot,Mid,IV,Breakeven,Breakeven %,Leverage,Cost/Contract,P(Profit),Edge Score,Ret 1M,Ret 3M,RSI,OI
0,STX,,2026-02-20,12,$400,93%,$429.32,$42.60,81.1%,$442.60,+3.1%,10x,"$4,260",39%,69,+50.9%,+54.6%,70,486
1,GEV,⭐,2026-02-20,12,$730,94%,$779.35,$60.00,50.8%,$790.00,+1.4%,13x,"$6,000",43%,68,+24.0%,+41.8%,71,810
2,GEV,⭐,2026-02-20,12,$735,94%,$779.35,$55.85,54.8%,$790.85,+1.5%,14x,"$5,585",43%,67,+24.0%,+41.8%,71,111
3,STX,,2026-02-20,12,$410,95%,$429.32,$35.95,79.6%,$445.95,+3.9%,12x,"$3,595",37%,67,+50.9%,+54.6%,70,116
4,LRCX,,2026-02-27,19,$220,95%,$231.01,$19.60,63.6%,$239.60,+3.7%,12x,"$1,960",38%,63,+15.0%,+42.7%,54,71
5,LRCX,,2026-02-20,12,$220,95%,$231.01,$17.23,63.9%,$237.22,+2.7%,13x,"$1,723",39%,63,+15.0%,+42.7%,54,"1,785"
6,CMI,,2026-02-20,12,$550,95%,$577.73,$30.80,33.9%,$580.80,+0.5%,19x,"$3,080",46%,63,+6.1%,+25.4%,50,155
7,AMAT,⭐,2026-02-20,12,$300,93%,$322.51,$30.07,70.2%,$330.07,+2.3%,11x,"$3,008",41%,63,+14.5%,+38.4%,48,"1,258"
8,WDC,,2026-02-20,12,$280,99%,$282.58,$19.65,86.7%,$299.65,+6.0%,14x,"$1,965",33%,62,+50.6%,+72.9%,66,438
9,WDC,,2026-02-20,12,$265,94%,$282.58,$27.20,82.5%,$292.20,+3.4%,10x,"$2,720",39%,62,+50.6%,+72.9%,66,227



### 💰 Quick Scenario — If Stock Pops

,,,,+5%,+10%,+15%
Ticker,Strike,DTE,Cost,,,
AMAT,300.000000,12,30.075000,+28%,+82%,+136%
CMI,550.000000,12,30.800000,+84%,+178%,+271%
GE,312.500000,12,12.700000,+93%,+220%,+346%
GEV,730.000000,12,60.000000,+47%,+112%,+177%
KLAC,1360.000000,12,103.500000,+50%,+120%,+189%
LRCX,220.000000,19,19.600000,+15%,+74%,+133%
META,625.000000,12,42.500000,+64%,+141%,+219%
STX,400.000000,12,42.600000,+19%,+70%,+120%


## 11. 🔄 Bounce Scanner — Which Pullbacks Are Ready to Reverse?

Several names on your watchlist have sold off hard (SNPS, UBER, ISRG, MSFT, AMZN, AVGO).  
Using a stack of technical indicators to score **mean-reversion probability**:

| Indicator                  | Signal                        | Logic                                       |
| -------------------------- | ----------------------------- | ------------------------------------------- |
| **RSI**                    | Oversold < 30                 | Snap-back due; < 25 = extreme               |
| **Distance from 200MA**    | Near/above support            | Holding long-term trend = strong floor      |
| **Bollinger Band %B**      | Below lower band              | Statistically stretched, reversion likely   |
| **MACD Histogram**         | Turning up from negative      | Momentum shifting from sellers to buyers    |
| **Volume spike**           | Above-avg volume on down days | Capitulation = near bottom                  |
| **Stochastic %K**          | < 20 + crossing up            | Classic oversold reversal signal            |
| **Distance from 52W High** | -15% to -25%                  | Pullback within healthy correction range    |
| **Support proximity**      | Near key MA levels            | Bouncing off support = higher prob reversal |


In [85]:
# ══════════════════════════════════════════════════════════════════
# Technical Bounce Scanner — deep dive on all tickers
# ══════════════════════════════════════════════════════════════════

bounce_rows = []

for ticker in ALL_TICKERS:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period="1y")
        if hist.empty or len(hist) < 60:
            continue

        closes = hist["Close"].dropna()
        highs = hist["High"].dropna()
        lows = hist["Low"].dropna()
        volumes = hist["Volume"].dropna()
        price = float(closes.iloc[-1])
        spot = spot_map.get(ticker, price)

        # ── RSI (14, Wilder's EWM) ──
        delta = closes.diff()
        gain = delta.clip(lower=0).ewm(com=13, adjust=False).mean()
        loss_s = (-delta.clip(upper=0)).ewm(com=13, adjust=False).mean()
        rs_val = gain.iloc[-1] / loss_s.iloc[-1] if loss_s.iloc[-1] > 0 else 100
        rsi = float(100 - (100 / (1 + rs_val)))

        # ── Stochastic %K / %D (14,3) ──
        low_14 = lows.rolling(14).min()
        high_14 = highs.rolling(14).max()
        stoch_k = (
            float(
                ((price - low_14.iloc[-1]) / (high_14.iloc[-1] - low_14.iloc[-1])) * 100
            )
            if (high_14.iloc[-1] - low_14.iloc[-1]) > 0
            else 50
        )
        stoch_k_series = ((closes - low_14) / (high_14 - low_14)) * 100
        stoch_d = float(stoch_k_series.rolling(3).mean().iloc[-1])
        stoch_cross_up = stoch_k > stoch_d and stoch_k < 30  # crossing up from oversold

        # ── Bollinger Bands (20, 2σ) ──
        bb_mid = closes.rolling(20).mean()
        bb_std = closes.rolling(20).std()
        bb_upper = bb_mid + 2 * bb_std
        bb_lower = bb_mid - 2 * bb_std
        bb_pct_b = (
            float((price - bb_lower.iloc[-1]) / (bb_upper.iloc[-1] - bb_lower.iloc[-1]))
            if (bb_upper.iloc[-1] - bb_lower.iloc[-1]) > 0
            else 0.5
        )
        below_lower_bb = price < bb_lower.iloc[-1]

        # ── MACD (12,26,9) ──
        ema12 = closes.ewm(span=12).mean()
        ema26 = closes.ewm(span=26).mean()
        macd_line = ema12 - ema26
        macd_signal = macd_line.ewm(span=9).mean()
        macd_hist = macd_line - macd_signal
        macd_hist_val = float(macd_hist.iloc[-1])
        macd_hist_prev = (
            float(macd_hist.iloc[-2]) if len(macd_hist) > 1 else macd_hist_val
        )
        macd_turning_up = (
            macd_hist_val > macd_hist_prev and macd_hist_val < 0
        )  # negative but improving

        # ── Moving Averages ──
        ma20 = float(bb_mid.iloc[-1]) if not pd.isna(bb_mid.iloc[-1]) else None
        ma50 = float(closes.rolling(50).mean().iloc[-1]) if len(closes) >= 50 else None
        ma200 = (
            float(closes.rolling(200).mean().iloc[-1]) if len(closes) >= 200 else None
        )
        dist_ma20 = (price - ma20) / ma20 if ma20 else np.nan
        dist_ma50 = (price - ma50) / ma50 if ma50 else np.nan
        dist_ma200 = (price - ma200) / ma200 if ma200 else np.nan

        # ── 52-week stats ──
        high_52w = float(closes.max())
        low_52w = float(closes.min())
        dist_high = (price - high_52w) / high_52w
        dist_low = (price - low_52w) / low_52w

        # ── Volume analysis ──
        avg_vol_20 = float(volumes.rolling(20).mean().iloc[-1])
        recent_vol = float(volumes.iloc[-5:].mean())  # last 5 days
        vol_ratio = recent_vol / avg_vol_20 if avg_vol_20 > 0 else 1.0

        # ── Returns ──
        ret_1w = float(price / closes.iloc[-5] - 1) if len(closes) > 5 else 0
        ret_1m = float(price / closes.iloc[-21] - 1) if len(closes) > 21 else 0
        ret_3m = float(price / closes.iloc[-63] - 1) if len(closes) > 63 else 0

        # ═══════════════════════════════════════════════════════
        # BOUNCE SCORE (0-100) — higher = more likely to reverse up
        # ═══════════════════════════════════════════════════════
        score = 0
        signals = []

        # RSI oversold (0-25 pts)
        if rsi < 20:
            score += 25
            signals.append(f"RSI extreme ({rsi:.0f})")
        elif rsi < 30:
            score += 20
            signals.append(f"RSI oversold ({rsi:.0f})")
        elif rsi < 40:
            score += 10
            signals.append(f"RSI weak ({rsi:.0f})")
        elif rsi > 65:
            score += 0  # already running, not a bounce play

        # Stochastic oversold + cross (0-15 pts)
        if stoch_cross_up:
            score += 15
            signals.append("Stoch cross up from oversold")
        elif stoch_k < 20:
            score += 10
            signals.append(f"Stoch oversold ({stoch_k:.0f})")

        # Bollinger Band position (0-15 pts)
        if below_lower_bb:
            score += 15
            signals.append("Below lower BB — stretched")
        elif bb_pct_b < 0.15:
            score += 10
            signals.append(f"Near lower BB (%B={bb_pct_b:.2f})")

        # MACD turning up from negative (0-12 pts)
        if macd_turning_up:
            score += 12
            signals.append("MACD histogram turning up")
        elif macd_hist_val > 0 and macd_hist_prev < 0:
            score += 10
            signals.append("MACD crossed positive")

        # Support proximity — near key MAs (0-12 pts)
        if ma200 and abs(dist_ma200) < 0.03:
            score += 12
            signals.append(f"At 200MA support ({dist_ma200:+.1%})")
        elif ma50 and abs(dist_ma50) < 0.03:
            score += 8
            signals.append(f"At 50MA ({dist_ma50:+.1%})")
        elif ma200 and dist_ma200 > 0:
            score += 5
            signals.append("Above 200MA")

        # 52W high distance — sweet spot is -10% to -25% (0-10 pts)
        if -0.25 < dist_high < -0.08:
            score += 10
            signals.append(f"Healthy pullback ({dist_high:.0%} from 52W high)")
        elif -0.35 < dist_high < -0.25:
            score += 6
            signals.append(f"Deep pullback ({dist_high:.0%} from 52W high)")

        # Volume capitulation signal (0-8 pts)
        if vol_ratio > 1.5 and ret_1w < -0.03:
            score += 8
            signals.append(f"Volume spike on sell-off ({vol_ratio:.1f}x avg)")
        elif vol_ratio > 1.2:
            score += 4
            signals.append(f"Elevated volume ({vol_ratio:.1f}x)")

        # Penalize if still in freefall (no stabilization)
        if ret_1w < -0.08:
            score -= 10
            signals.append("⚠ Still falling sharply this week")

        # Bonus: golden cross still intact despite pullback = strong base
        hv = hv_momentum.get(ticker, {})
        if hv.get("golden_cross") and rsi < 40:
            score += 5
            signals.append("Golden cross intact on pullback")

        score = max(0, min(100, score))

        bounce_rows.append(
            {
                "Ticker": ticker,
                "Conviction": "⭐" if ticker in CONVICTION else "",
                "Spot": spot,
                "RSI": rsi,
                "Stoch %K": stoch_k,
                "Stoch %D": stoch_d,
                "BB %B": bb_pct_b,
                "MACD Hist": macd_hist_val,
                "MACD Turning": "✅" if macd_turning_up else "—",
                "Dist 200MA": dist_ma200,
                "Dist 50MA": dist_ma50,
                "Dist 52W High": dist_high,
                "Dist 52W Low": dist_low,
                "Ret 1W": ret_1w,
                "Ret 1M": ret_1m,
                "Ret 3M": ret_3m,
                "Vol Ratio": vol_ratio,
                "Bounce Score": score,
                "Signals": " | ".join(signals) if signals else "No strong signals",
            }
        )

        status = "🟢" if score >= 50 else "🟡" if score >= 30 else "⚪"
        print(
            f"  {status} {ticker}: Bounce={score} RSI={rsi:.0f} Stoch={stoch_k:.0f} BB%B={bb_pct_b:.2f} 1M={ret_1m:+.1%}"
        )

    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")

bounce_df = pd.DataFrame(bounce_rows)
bounce_df = bounce_df.sort_values("Bounce Score", ascending=False)

print(f"\n✅ Scanned {len(bounce_df)} tickers for bounce setups")

  ⚪ WDC: Bounce=9 RSI=64 Stoch=82 BB%B=0.85 1M=+50.6%
  ⚪ GEV: Bounce=5 RSI=68 Stoch=89 BB%B=0.95 1M=+24.0%
  ⚪ STX: Bounce=5 RSI=65 Stoch=78 BB%B=0.78 1M=+50.9%
  ⚪ LRCX: Bounce=17 RSI=58 Stoch=56 BB%B=0.64 1M=+15.0%
  ⚪ AMAT: Bounce=17 RSI=57 Stoch=61 BB%B=0.58 1M=+14.5%
  ⚪ TSM: Bounce=17 RSI=62 Stoch=97 BB%B=1.03 1M=+9.7%
  ⚪ GE: Bounce=15 RSI=59 Stoch=91 BB%B=0.74 1M=+2.1%
  ⚪ CMI: Bounce=9 RSI=54 Stoch=56 BB%B=0.53 1M=+6.1%
  ⚪ KLAC: Bounce=27 RSI=52 Stoch=39 BB%B=0.41 1M=+8.9%
  🟢 SNPS: Bounce=61 RSI=34 Stoch=18 BB%B=0.12 1M=-17.0%
  ⚪ META: Bounce=18 RSI=49 Stoch=43 BB%B=0.52 1M=+2.4%
  🟡 UBER: Bounce=49 RSI=32 Stoch=21 BB%B=0.03 1M=-14.6%
  🟢 ISRG: Bounce=51 RSI=29 Stoch=30 BB%B=0.22 1M=-16.6%
  🟢 MSFT: Bounce=67 RSI=29 Stoch=10 BB%B=0.08 1M=-16.1%
  🟢 AMZN: Bounce=63 RSI=29 Stoch=21 BB%B=-0.28 1M=-14.6%
  🟡 AVGO: Bounce=31 RSI=49 Stoch=75 BB%B=0.51 1M=+0.1%

✅ Scanned 16 tickers for bounce setups


### Bounce Rankings & LEAPS Entry Points


In [78]:
if bounce_df.empty:
    display(Markdown("❌ No bounce data."))
else:
    # ── Full ranking table ──
    display(Markdown("### 🔄 Bounce Score Rankings — All Tickers"))
    rank_cols = [
        "Ticker",
        "Conviction",
        "Spot",
        "Bounce Score",
        "RSI",
        "Stoch %K",
        "BB %B",
        "MACD Turning",
        "Dist 200MA",
        "Dist 52W High",
        "Ret 1W",
        "Ret 1M",
        "Ret 3M",
        "Vol Ratio",
        "Signals",
    ]
    ra = [c for c in rank_cols if c in bounce_df.columns]
    display(
        bounce_df[ra]
        .style.format(
            {
                "Spot": "${:,.2f}",
                "Bounce Score": "{:.0f}",
                "RSI": "{:.0f}",
                "Stoch %K": "{:.0f}",
                "BB %B": "{:.2f}",
                "Dist 200MA": "{:+.1%}",
                "Dist 52W High": "{:.0%}",
                "Ret 1W": "{:+.1%}",
                "Ret 1M": "{:+.1%}",
                "Ret 3M": "{:+.1%}",
                "Vol Ratio": "{:.1f}x",
            },
            na_rep="—",
        )
        .background_gradient(subset=["Bounce Score"], cmap="RdYlGn", vmin=0, vmax=80)
        .background_gradient(
            subset=["RSI"],
            cmap="RdYlGn_r",
            vmin=20,
            vmax=70,  # low RSI = green (oversold = good for bounce)
        )
        .set_caption("Sorted by Bounce Score — higher = more reversal signals firing")
    )

    # ── Chart: Bounce Score vs RSI with Dist from 52W High as size ──
    plot_b = bounce_df.copy()
    plot_b["abs_dist_high"] = plot_b["Dist 52W High"].abs() * 100
    plot_b["is_conviction"] = plot_b["Ticker"].isin(CONVICTION)

    fig_b = px.scatter(
        plot_b,
        x="RSI",
        y="Bounce Score",
        text="Ticker",
        size="abs_dist_high",
        size_max=30,
        color="is_conviction",
        color_discrete_map={True: "#1a7431", False: "#4C6E91"},
        title="Bounce Score vs RSI — Bigger Bubble = Further from 52W High",
        labels={"is_conviction": "Conviction"},
    )
    fig_b.update_traces(textposition="top center")
    fig_b.add_vline(
        x=30,
        line_dash="dot",
        line_color="red",
        opacity=0.5,
        annotation_text="RSI Oversold",
    )
    fig_b.add_hline(y=40, line_dash="dot", line_color="gray", opacity=0.5)
    fig_b.add_annotation(
        x=22,
        y=65,
        text="🎯 Bounce Zone",
        showarrow=False,
        font=dict(size=14, color="green"),
    )
    fig_b.show()

    # ── Tier the bounce candidates ──
    high_bounce = bounce_df[bounce_df["Bounce Score"] >= 45].copy()
    mid_bounce = bounce_df[
        (bounce_df["Bounce Score"] >= 25) & (bounce_df["Bounce Score"] < 45)
    ].copy()
    low_bounce = bounce_df[bounce_df["Bounce Score"] < 25].copy()

    if not high_bounce.empty:
        display(Markdown("\n### 🟢 Strong Bounce Setups"))
        for _, r in high_bounce.iterrows():
            display(
                Markdown(
                    f"**{r['Ticker']}** {r['Conviction']} — Bounce Score: **{r['Bounce Score']:.0f}** | "
                    f"RSI: {r['RSI']:.0f} | Stoch: {r['Stoch %K']:.0f} | "
                    f"BB%B: {r['BB %B']:.2f} | MACD: {r['MACD Turning']} | "
                    f"1M: {r['Ret 1M']:+.1%} | From 52W High: {r['Dist 52W High']:.0%}\n"
                    f"> Signals: *{r['Signals']}*"
                )
            )

    if not mid_bounce.empty:
        display(Markdown("\n### 🟡 Developing — Watch for Confirmation"))
        for _, r in mid_bounce.iterrows():
            display(
                Markdown(
                    f"**{r['Ticker']}** {r['Conviction']} — Bounce Score: **{r['Bounce Score']:.0f}** | "
                    f"RSI: {r['RSI']:.0f} | 1M: {r['Ret 1M']:+.1%} | "
                    f"From 52W High: {r['Dist 52W High']:.0%}\n"
                    f"> Signals: *{r['Signals']}*"
                )
            )

    # ── Best LEAPS for the top bounce candidates ──
    bounce_tickers = bounce_df[bounce_df["Bounce Score"] >= 30]["Ticker"].tolist()
    if bounce_tickers and not leaps_df.empty:
        display(Markdown("\n---\n### 💰 LEAPS Entry Points for Top Bounce Candidates"))
        display(
            Markdown(
                "*If you believe the bounce is coming, these are the options to express that view:*"
            )
        )

        bounce_leaps = scored_df[scored_df["Ticker"].isin(bounce_tickers)].copy()
        if not bounce_leaps.empty:
            # Best option per ticker: balance of edge + P(profit)
            bounce_leaps["combined"] = (
                bounce_leaps["Edge Score"] * 0.6
                + bounce_leaps["P(Profit)"].fillna(0) * 100 * 0.4
            )
            bl_top = (
                bounce_leaps.sort_values("combined", ascending=False)
                .groupby("Ticker")
                .head(2)
                .reset_index(drop=True)
            )

            # Merge bounce score
            bl_top = bl_top.merge(
                bounce_df[["Ticker", "Bounce Score", "RSI"]].rename(
                    columns={"RSI": "RSI_bounce"}
                ),
                on="Ticker",
                how="left",
            )
            bl_top = bl_top.sort_values("Bounce Score", ascending=False)

            bl_cols = [
                "Ticker",
                "Conviction",
                "Bounce Score",
                "Expiry",
                "DTE",
                "Strike",
                "Moneyness",
                "Spot",
                "Mid",
                "IV",
                "Breakeven %",
                "Leverage",
                "Cost/Contract",
                "P(Profit)",
                "Edge Score",
            ]
            bla = [c for c in bl_cols if c in bl_top.columns]
            display(
                bl_top[bla]
                .style.format(
                    {
                        "Bounce Score": "{:.0f}",
                        "Strike": "${:,.0f}",
                        "Spot": "${:,.2f}",
                        "Mid": "${:,.2f}",
                        "Moneyness": "{:.0%}",
                        "IV": "{:.1%}",
                        "Breakeven %": "{:+.1%}",
                        "Leverage": "{:.1f}x",
                        "Cost/Contract": "${:,.0f}",
                        "P(Profit)": "{:.0%}",
                        "Edge Score": "{:.0f}",
                    },
                    na_rep="—",
                )
                .background_gradient(
                    subset=["Bounce Score"], cmap="RdYlGn", vmin=20, vmax=70
                )
                .set_caption(
                    "LEAPS calls for bounce candidates — sorted by Bounce Score"
                )
            )

### 🔄 Bounce Score Rankings — All Tickers

,Ticker,Conviction,Spot,Bounce Score,RSI,Stoch %K,BB %B,MACD Turning,Dist 200MA,Dist 52W High,Ret 1W,Ret 1M,Ret 3M,Vol Ratio,Signals
9,SNPS,,$426.88,71,23,18,0.12,✅,-14.2%,-34%,-6.8%,-17.0%,+7.8%,1.6x,RSI oversold (23) | Stoch cross up from oversold | Near lower BB (%B=0.12) | MACD histogram turning up | Deep pullback (-34% from 52W high) | Volume spike on sell-off (1.6x avg)
14,AMZN,,$210.32,63,28,21,-0.28,—,-5.8%,-17%,-13.4%,-14.6%,-13.5%,1.7x,RSI oversold (28) | Stoch cross up from oversold | Below lower BB — stretched | Healthy pullback (-17% from 52W high) | Volume spike on sell-off (1.7x avg) | ⚠ Still falling sharply this week | Golden cross intact on pullback
11,UBER,,$74.77,59,24,21,0.03,—,-16.1%,-25%,-7.5%,-14.6%,-18.8%,1.5x,RSI oversold (24) | Stoch cross up from oversold | Near lower BB (%B=0.03) | Deep pullback (-25% from 52W high) | Volume spike on sell-off (1.5x avg)
13,MSFT,,$401.14,57,31,10,0.08,✅,-17.4%,-26%,-5.3%,-16.1%,-19.2%,1.3x,RSI weak (31) | Stoch cross up from oversold | Near lower BB (%B=0.08) | MACD histogram turning up | Deep pullback (-26% from 52W high) | Elevated volume (1.3x)
12,ISRG,,$488.15,51,22,30,0.22,✅,-5.5%,-20%,-1.7%,-16.6%,-10.9%,1.2x,RSI oversold (22) | MACD histogram turning up | Healthy pullback (-20% from 52W high) | Elevated volume (1.2x) | Golden cross intact on pullback
15,AVGO,⭐,$332.92,31,40,75,0.51,✅,+8.0%,-19%,+0.5%,+0.1%,-6.2%,1.2x,MACD histogram turning up | Above 200MA | Healthy pullback (-19% from 52W high) | Elevated volume (1.2x)
8,KLAC,,"$1,442.95",27,43,39,0.41,✅,+39.0%,-14%,+2.3%,+8.9%,+19.8%,1.1x,MACD histogram turning up | Above 200MA | Healthy pullback (-14% from 52W high)
10,META,,$661.46,18,59,43,0.52,—,-3.4%,-16%,-6.4%,+2.4%,+7.0%,0.8x,At 50MA (+0.8%) | Healthy pullback (-16% from 52W high)
4,AMAT,⭐,$322.51,17,48,61,0.58,✅,+52.2%,-6%,-1.8%,+14.5%,+38.4%,1.2x,MACD histogram turning up | Above 200MA
5,TSM,,$348.85,17,53,97,1.03,✅,+35.0%,0%,+2.2%,+9.7%,+20.9%,1.0x,MACD histogram turning up | Above 200MA



### 🟢 Strong Bounce Setups

**SNPS**  — Bounce Score: **71** | RSI: 23 | Stoch: 18 | BB%B: 0.12 | MACD: ✅ | 1M: -17.0% | From 52W High: -34%
> Signals: *RSI oversold (23) | Stoch cross up from oversold | Near lower BB (%B=0.12) | MACD histogram turning up | Deep pullback (-34% from 52W high) | Volume spike on sell-off (1.6x avg)*

**AMZN**  — Bounce Score: **63** | RSI: 28 | Stoch: 21 | BB%B: -0.28 | MACD: — | 1M: -14.6% | From 52W High: -17%
> Signals: *RSI oversold (28) | Stoch cross up from oversold | Below lower BB — stretched | Healthy pullback (-17% from 52W high) | Volume spike on sell-off (1.7x avg) | ⚠ Still falling sharply this week | Golden cross intact on pullback*

...

**AVGO** ⭐ — Bounce Score: **31** | RSI: 40 | 1M: +0.1% | From 52W High: -19%
> Signals: *MACD histogram turning up | Above 200MA | Healthy pullback (-19% from 52W high) | Elevated volume (1.2x)*

**KLAC**  — Bounce Score: **27** | RSI: 43 | 1M: +8.9% | From 52W High: -14%
> Signals: *MACD histogram turning up | Above 200MA | Healthy pullback (-14% from 52W high)*


---
### 💰 LEAPS Entry Points for Top Bounce Candidates

*If you believe the bounce is coming, these are the options to express that view:*

,Ticker,Conviction,Bounce Score,Expiry,DTE,Strike,Moneyness,Spot,Mid,IV,Breakeven %,Leverage,Cost/Contract,P(Profit),Edge Score
9,SNPS,,71,2026-09-18,222,$350,82%,$426.88,$112.30,55.9%,+8.3%,3.8x,"$11,230",37%,45
10,SNPS,,71,2026-09-18,222,$360,84%,$426.88,$105.80,55.3%,+9.1%,4.0x,"$10,580",36%,45
3,AMZN,,63,2026-05-15,96,$175,83%,$210.32,$41.38,48.8%,+2.9%,5.1x,"$4,138",42%,46
2,AMZN,,63,2026-06-18,130,$175,83%,$210.32,$43.15,46.1%,+3.7%,4.9x,"$4,315",41%,47
7,UBER,,59,2026-06-18,130,$60,80%,$74.77,$17.27,50.8%,+3.4%,4.3x,"$1,727",42%,43
6,UBER,,59,2026-06-18,130,$65,87%,$74.77,$13.15,47.0%,+4.5%,5.7x,"$1,315",40%,44
0,MSFT,,57,2026-06-18,130,$350,87%,$401.14,$64.72,38.5%,+3.4%,6.2x,"$6,472",42%,55
1,MSFT,,57,2026-05-15,96,$350,87%,$401.14,$62.08,40.4%,+2.7%,6.5x,"$6,208",43%,54
11,ISRG,,51,2026-07-17,159,$400,82%,$488.15,$105.10,44.3%,+3.5%,4.6x,"$10,510",42%,40
5,ISRG,,51,2026-06-18,130,$400,82%,$488.15,$102.00,44.4%,+2.8%,4.8x,"$10,200",43%,43


## 📋 Final Call Picks — Unified by DTE Bucket

All signals combined: **Edge Score** (BSM probability, IV vs HV, momentum, style) + **Bounce Score** (RSI, Stochastic, Bollinger, MACD, support levels).

| Bucket             | DTE   | Strategy                         | Thesis                            |
| ------------------ | ----- | -------------------------------- | --------------------------------- |
| ⚡ **Short-Term**  | <30   | Momentum swings, bounce snaps    | High leverage, ride a quick move  |
| 🔄 **Medium-Term** | 30–90 | Bounce recovery, earnings setups | Time for oversold names to revert |
| 🏗️ **LEAPS**       | 300+  | Conviction holds, deep-ITM       | Stock replacement, long runway    |


In [86]:
# ══════════════════════════════════════════════════════════════════
# STEP 1 — Fetch medium-term (30-90 DTE) options to fill the gap
#          Also fetch <30 DTE for bounce tickers we didn't swing-scan
# ══════════════════════════════════════════════════════════════════

MED_DTE_MIN, MED_DTE_MAX = 30, 90
SHORT_DTE_MAX = 30
MED_MONEYNESS = (0.88, 1.05)

med_rows = []
short_extra_rows = []  # <30 DTE for non-momentum (bounce) tickers

# Which tickers are missing from swing_df's <30 DTE?
swing_tickers_set = set(swing_df["Ticker"].unique()) if not swing_df.empty else set()
bounce_only = [t for t in ALL_TICKERS if t not in swing_tickers_set]

print("Fetching medium-term (30-90 DTE) + short-term gap fills...\n")

for ticker in ALL_TICKERS:
    spot = spot_map.get(ticker)
    if not spot:
        continue
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        all_exps = t.options or []
        today = datetime.now().date()

        target_exps = []
        for exp_str in all_exps:
            try:
                exp_date = datetime.strptime(exp_str, "%Y-%m-%d").date()
                dte = (exp_date - today).days
                # Medium-term for ALL tickers
                if MED_DTE_MIN <= dte <= MED_DTE_MAX:
                    target_exps.append((exp_str, dte, "med"))
                # Short-term for bounce-only tickers (not already in swing_df)
                elif dte < SHORT_DTE_MAX and dte >= 7 and ticker in bounce_only:
                    target_exps.append((exp_str, dte, "short"))
            except ValueError:
                continue

        if not target_exps:
            print(f"  ⚪ {ticker}: no exps in target range")
            continue

        # Take up to 2 per bucket
        med_exps = sorted(
            [e for e in target_exps if e[2] == "med"], key=lambda x: x[1]
        )[:2]
        short_exps = sorted(
            [e for e in target_exps if e[2] == "short"], key=lambda x: x[1]
        )[:1]

        for exp_str, dte, bucket in med_exps + short_exps:
            time.sleep(RATE_LIMIT_SLEEP)
            chain = t.option_chain(exp_str)
            calls = chain.calls.copy()
            if calls.empty:
                continue

            calls["mid"] = (calls["bid"] + calls["ask"]) / 2
            calls.loc[calls["mid"] <= 0, "mid"] = calls["lastPrice"]
            calls["moneyness"] = calls["strike"] / spot
            calls["spread_pct"] = np.where(
                calls["mid"] > 0, (calls["ask"] - calls["bid"]) / calls["mid"], np.nan
            )

            mask = (
                (calls["moneyness"] >= MED_MONEYNESS[0])
                & (calls["moneyness"] <= MED_MONEYNESS[1])
                & (calls["mid"] > 0.5)
            )
            filtered = calls[mask].copy()
            if filtered.empty:
                continue

            for _, opt in filtered.iterrows():
                iv = safe_float(opt.get("impliedVolatility"))
                intrinsic = max(spot - opt["strike"], 0)
                breakeven = opt["strike"] + opt["mid"]
                breakeven_pct = (breakeven - spot) / spot
                leverage = spot / opt["mid"] if opt["mid"] > 0 else 0
                p_profit = bsm_prob_profit(spot, breakeven, iv, dte)
                oi = safe_float(opt.get("openInterest", 0))

                # Edge score — aligned with score_option() weights
                hv = hv_momentum.get(ticker, {})
                edge = 0
                n = 0
                if not np.isnan(p_profit):
                    edge += p_profit * 100 * 0.25
                    n += 0.25
                ret_1m = hv.get("ret_1m", 0)
                ret_3m = hv.get("ret_3m", 0)
                mom = np.clip(ret_1m * 200, -30, 40) + np.clip(ret_3m * 100, -20, 30)
                rsi_val = hv.get("rsi", 50)
                if 45 < rsi_val < 72:
                    mom += 15
                elif rsi_val >= 72:
                    mom += 5
                if hv.get("above_50ma"):
                    mom += 5
                if hv.get("above_200ma"):
                    mom += 5
                if hv.get("golden_cross"):
                    mom += 5
                edge += np.clip(mom, 0, 100) * 0.20
                n += 0.20
                be_score = np.clip((0.15 - breakeven_pct) / 0.20 * 100, 0, 100)
                edge += be_score * 0.15
                n += 0.15
                hv30 = hv.get("hv_30", np.nan)
                if not np.isnan(iv) and not np.isnan(hv30) and hv30 > 0:
                    ratio = iv / hv30
                    if ratio < 0.9:
                        iv_s = 90
                    elif ratio < 1.1:
                        iv_s = 70
                    elif ratio < 1.3:
                        iv_s = 45
                    else:
                        iv_s = max(0, 30 - (ratio - 1.3) * 50)
                    edge += iv_s * 0.15
                    n += 0.15
                # Style match (moneyness proximity) — 15%
                ideal_mon = STYLE_PREFS.get("ideal_moneyness_leaps", 0.90)
                mon_dist = abs(opt["moneyness"] - ideal_mon)
                style_s = max(0, 100 - mon_dist * 500)
                if opt["moneyness"] < 0.90:
                    style_s = min(100, style_s + 10)
                edge += style_s * 0.15
                n += 0.15
                sp = opt["spread_pct"] if not np.isnan(opt["spread_pct"]) else 0.5
                liq = min(
                    100,
                    (40 if oi > 500 else 25 if oi > 100 else 10 if oi > 20 else 0)
                    + (
                        60 if sp < 0.05 else 40 if sp < 0.10 else 20 if sp < 0.20 else 0
                    ),
                )
                edge += liq * 0.10
                n += 0.10
                edge = edge / n if n > 0 else 0

                row_data = {
                    "Ticker": ticker,
                    "Conviction": "⭐" if ticker in CONVICTION else "",
                    "Expiry": exp_str,
                    "DTE": dte,
                    "Strike": opt["strike"],
                    "Moneyness": opt["moneyness"],
                    "Spot": spot,
                    "Mid": opt["mid"],
                    "IV": iv,
                    "Breakeven": breakeven,
                    "Breakeven %": breakeven_pct,
                    "Leverage": leverage,
                    "Cost/Contract": opt["mid"] * 100,
                    "P(Profit)": p_profit,
                    "Edge Score": edge,
                    "OI": oi,
                    "Spread %": opt["spread_pct"],
                    "Ret 1M": hv.get("ret_1m", np.nan),
                    "Ret 3M": hv.get("ret_3m", np.nan),
                    "RSI": hv.get("rsi", np.nan),
                }
                if bucket == "med":
                    med_rows.append(row_data)
                else:
                    short_extra_rows.append(row_data)

        n_med = len([r for r in med_rows if r["Ticker"] == ticker])
        n_short = len([r for r in short_extra_rows if r["Ticker"] == ticker])
        print(f"  ✅ {ticker}: {n_med} med-term + {n_short} short-term")

    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")

med_df = pd.DataFrame(med_rows)
short_extra_df = pd.DataFrame(short_extra_rows)
print(
    f"\n✅ Medium-term (30-90 DTE): {len(med_df)} options across {med_df['Ticker'].nunique() if not med_df.empty else 0} tickers"
)
print(f"✅ Short-term gap fills: {len(short_extra_df)} options")


# ══════════════════════════════════════════════════════════════════
# STEP 2 — Combine ALL options into unified DataFrame
# ══════════════════════════════════════════════════════════════════

common_cols = [
    "Ticker",
    "Conviction",
    "Expiry",
    "DTE",
    "Strike",
    "Moneyness",
    "Spot",
    "Mid",
    "IV",
    "Breakeven",
    "Breakeven %",
    "Leverage",
    "Cost/Contract",
    "P(Profit)",
    "Edge Score",
    "OI",
    "Spread %",
    "Ret 1M",
    "Ret 3M",
    "RSI",
]

frames = []

# Short-term from swing_df
if not swing_df.empty:
    sw = swing_df.copy()
    for c in common_cols:
        if c not in sw.columns:
            sw[c] = np.nan
    frames.append(sw[common_cols])

# Short-term gap fills (bounce tickers)
if not short_extra_df.empty:
    for c in common_cols:
        if c not in short_extra_df.columns:
            short_extra_df[c] = np.nan
    frames.append(short_extra_df[common_cols])

# Medium-term
if not med_df.empty:
    for c in common_cols:
        if c not in med_df.columns:
            med_df[c] = np.nan
    frames.append(med_df[common_cols])

# LEAPS from scored_df
if not scored_df.empty:
    sc = scored_df.copy()
    for c in common_cols:
        if c not in sc.columns:
            sc[c] = np.nan
    frames.append(sc[common_cols])

unified_df = pd.concat(frames, ignore_index=True)


# DTE bucket labels
def dte_bucket(d):
    if d < 30:
        return "⚡ <30 DTE"
    elif d <= 90:
        return "🔄 30-90 DTE"
    else:
        return "🏗️ 300+ DTE"


unified_df["Bucket"] = unified_df["DTE"].apply(dte_bucket)

# Merge bounce scores
if not bounce_df.empty:
    bounce_merge = bounce_df[["Ticker", "Bounce Score", "Signals"]].copy()
    unified_df = unified_df.merge(bounce_merge, on="Ticker", how="left")
    unified_df["Bounce Score"] = unified_df["Bounce Score"].fillna(0)
else:
    unified_df["Bounce Score"] = 0
    unified_df["Signals"] = ""


# Combined rank score: Edge + Bounce bonus
# For short/med term, bounce matters more. For LEAPS, edge matters more.
def combined_score(row):
    edge = row["Edge Score"] if not np.isnan(row["Edge Score"]) else 0
    bounce = row["Bounce Score"]
    if row["Bucket"] == "⚡ <30 DTE":
        return edge * 0.55 + bounce * 0.45
    elif row["Bucket"] == "🔄 30-90 DTE":
        return edge * 0.50 + bounce * 0.50
    else:
        return edge * 0.70 + bounce * 0.30


unified_df["Combo Score"] = unified_df.apply(combined_score, axis=1)

# Drop exact duplicates (same ticker/strike/expiry)
unified_df = unified_df.drop_duplicates(
    subset=["Ticker", "Strike", "Expiry"], keep="first"
)

print(f"\n📊 Unified universe: {len(unified_df)} options")
for b in ["⚡ <30 DTE", "🔄 30-90 DTE", "🏗️ 300+ DTE"]:
    sub = unified_df[unified_df["Bucket"] == b]
    print(f"  {b}: {len(sub)} options across {sub['Ticker'].nunique()} tickers")

Fetching medium-term (30-90 DTE) + short-term gap fills...

  ✅ WDC: 14 med-term + 0 short-term
  ✅ GEV: 29 med-term + 0 short-term
  ✅ STX: 17 med-term + 0 short-term
  ✅ LRCX: 9 med-term + 0 short-term
  ✅ AMAT: 7 med-term + 0 short-term
  ✅ TSM: 16 med-term + 0 short-term
  ✅ GE: 10 med-term + 0 short-term
  ✅ CMI: 17 med-term + 0 short-term
  ✅ KLAC: 36 med-term + 0 short-term
  ✅ SNPS: 7 med-term + 7 short-term
  ✅ META: 34 med-term + 0 short-term
  ✅ UBER: 6 med-term + 10 short-term
  ✅ ISRG: 21 med-term + 17 short-term
  ✅ MSFT: 21 med-term + 14 short-term
  ✅ AMZN: 9 med-term + 8 short-term
  ✅ AVGO: 15 med-term + 16 short-term

✅ Medium-term (30-90 DTE): 268 options across 16 tickers
✅ Short-term gap fills: 72 options

📊 Unified universe: 1061 options
  ⚡ <30 DTE: 257 options across 16 tickers
  🔄 30-90 DTE: 268 options across 16 tickers
  🏗️ 300+ DTE: 536 options across 16 tickers


In [87]:
# ══════════════════════════════════════════════════════════════════
# FINAL PICKS — Best calls per DTE bucket
# ══════════════════════════════════════════════════════════════════

display(Markdown("---\n# 🎯 Final Call Picks\n"))

pick_cols = [
    "Ticker",
    "Conviction",
    "Bucket",
    "Expiry",
    "DTE",
    "Strike",
    "Moneyness",
    "Spot",
    "Mid",
    "IV",
    "Breakeven %",
    "Leverage",
    "Cost/Contract",
    "P(Profit)",
    "Edge Score",
    "Bounce Score",
    "Combo Score",
]

buckets = [
    (
        "⚡ <30 DTE",
        "⚡ Short-Term Swings — Momentum & Snap-Back Plays",
        "Quick directional bets. High leverage, tight risk. Look for RSI reversals + momentum continuation.",
    ),
    (
        "🔄 30-90 DTE",
        "🔄 Medium-Term — Bounce Recovery & Earnings Setups",
        "Give oversold names 1-3 months to revert. Best risk/reward for bounce thesis.",
    ),
    (
        "🏗️ 300+ DTE",
        "🏗️ LEAPS — Conviction Holds & Stock Replacement",
        "Deep ITM for delta exposure, ATM for leverage. Long runway = time to be right.",
    ),
]

all_picks = []  # collect for summary

for bucket_key, title, description in buckets:
    sub = unified_df[unified_df["Bucket"] == bucket_key].copy()
    if sub.empty:
        display(Markdown(f"\n### {title}\n*No options in this bucket.*"))
        continue

    # Pick top 2 per ticker, then top 15 overall
    top = (
        sub.sort_values("Combo Score", ascending=False)
        .groupby("Ticker")
        .head(2)
        .sort_values("Combo Score", ascending=False)
        .head(15)
        .reset_index(drop=True)
    )

    display(Markdown(f"\n### {title}"))
    display(Markdown(f"*{description}*"))

    avail = [c for c in pick_cols if c in top.columns]
    styled = (
        top[avail]
        .style.format(
            {
                "Strike": "${:,.0f}",
                "Spot": "${:,.2f}",
                "Mid": "${:,.2f}",
                "Moneyness": "{:.0%}",
                "IV": "{:.1%}",
                "Breakeven %": "{:+.1%}",
                "Leverage": "{:.0f}x",
                "Cost/Contract": "${:,.0f}",
                "P(Profit)": "{:.0%}",
                "Edge Score": "{:.0f}",
                "Bounce Score": "{:.0f}",
                "Combo Score": "{:.0f}",
            },
            na_rep="—",
        )
        .background_gradient(subset=["Combo Score"], cmap="RdYlGn", vmin=20, vmax=70)
    )
    if "P(Profit)" in avail:
        styled = styled.background_gradient(
            subset=["P(Profit)"], cmap="RdYlGn", vmin=0.15, vmax=0.65
        )
    if "Bounce Score" in avail:
        styled = styled.background_gradient(
            subset=["Bounce Score"], cmap="Blues", vmin=0, vmax=70
        )

    display(styled.set_caption(f"Top picks — {bucket_key} | Sorted by Combo Score"))

    # Scenario P&L for this bucket
    best_per_ticker = top.groupby("Ticker").first().reset_index().head(6)
    if bucket_key == "⚡ <30 DTE":
        moves = [0.03, 0.05, 0.08, 0.10]
    elif bucket_key == "🔄 30-90 DTE":
        moves = [0.05, 0.10, 0.15, 0.20]
    else:
        moves = [0.10, 0.20, 0.30, 0.50]

    scen = []
    for _, r in best_per_ticker.iterrows():
        for m in moves:
            new_spot = r["Spot"] * (1 + m)
            payout = max(new_spot - r["Strike"], 0)
            ret = (payout - r["Mid"]) / r["Mid"] if r["Mid"] > 0 else 0
            scen.append(
                {
                    "Ticker": r["Ticker"],
                    "Strike": r["Strike"],
                    "DTE": r["DTE"],
                    "Cost": f"${r['Mid']:,.2f}",
                    "Move": m,
                    "Return": ret,
                }
            )
    if scen:
        scen_df = pd.DataFrame(scen)
        piv = scen_df.pivot_table(
            index=["Ticker", "Strike", "DTE", "Cost"], columns="Move", values="Return"
        )
        piv.columns = [f"+{c:.0%}" for c in piv.columns]
        display(
            piv.style.format("{:+.0%}", na_rep="—")
            .background_gradient(cmap="RdYlGn", vmin=-0.5, vmax=3.0, axis=None)
            .set_caption(f"Return % if stock rallies — {bucket_key}")
        )

    all_picks.append(top)

# ══════════════════════════════════════════════════════════════════
# SUMMARY — One-page cheat sheet
# ══════════════════════════════════════════════════════════════════

display(Markdown("\n---\n## 🗒️ Cheat Sheet — Top Pick Per Bucket Per Ticker\n"))

summary_rows = []
for bucket_key, _, _ in buckets:
    sub = unified_df[unified_df["Bucket"] == bucket_key].copy()
    if sub.empty:
        continue
    best = (
        sub.sort_values("Combo Score", ascending=False)
        .groupby("Ticker")
        .first()
        .reset_index()
    )
    best["Bucket"] = bucket_key
    summary_rows.append(best)

if summary_rows:
    cheat = pd.concat(summary_rows, ignore_index=True)
    cheat = cheat.sort_values(["Ticker", "DTE"])

    cheat["Call"] = cheat.apply(
        lambda r: f"${r['Strike']:,.0f} {r['Expiry']} ({r['DTE']}d)", axis=1
    )
    cheat["Entry"] = cheat["Mid"].apply(lambda x: f"${x:,.2f}")
    cheat["B/E"] = cheat["Breakeven %"].apply(
        lambda x: f"{x:+.1%}" if not np.isnan(x) else "—"
    )

    summary_cols = [
        "Ticker",
        "Conviction",
        "Bucket",
        "Call",
        "Entry",
        "B/E",
        "Leverage",
        "P(Profit)",
        "Edge Score",
        "Bounce Score",
        "Combo Score",
    ]
    sa = [c for c in summary_cols if c in cheat.columns]

    display(
        cheat[sa]
        .style.format(
            {
                "Leverage": "{:.0f}x",
                "P(Profit)": "{:.0%}",
                "Edge Score": "{:.0f}",
                "Bounce Score": "{:.0f}",
                "Combo Score": "{:.0f}",
            },
            na_rep="—",
        )
        .background_gradient(subset=["Combo Score"], cmap="RdYlGn", vmin=20, vmax=70)
        .set_caption("Best single call per ticker per time horizon")
    )

# ── Visual: Scatter of all top picks ──
if all_picks:
    all_top = pd.concat(all_picks, ignore_index=True)
    fig_final = px.scatter(
        all_top,
        x="DTE",
        y="Combo Score",
        text="Ticker",
        color="Bucket",
        size="Leverage",
        size_max=25,
        color_discrete_map={
            "⚡ <30 DTE": "#e74c3c",
            "🔄 30-90 DTE": "#f39c12",
            "🏗️ 300+ DTE": "#2ecc71",
        },
        title="Final Picks — Combo Score vs DTE (size = leverage)",
        labels={"Combo Score": "Combo Score (Edge + Bounce)"},
        log_x=True,
    )
    fig_final.update_traces(textposition="top center", textfont_size=9)
    fig_final.update_layout(height=500)
    fig_final.show()

---
# 🎯 Final Call Picks



### ⚡ Short-Term Swings — Momentum & Snap-Back Plays

*Quick directional bets. High leverage, tight risk. Look for RSI reversals + momentum continuation.*

,Ticker,Conviction,Bucket,Expiry,DTE,Strike,Moneyness,Spot,Mid,IV,Breakeven %,Leverage,Cost/Contract,P(Profit),Edge Score,Bounce Score,Combo Score
0,MSFT,,⚡ <30 DTE,2026-02-20,12,$360,90%,$401.14,$42.45,51.5%,+0.3%,9x,"$4,245",47%,49,67,57
1,MSFT,,⚡ <30 DTE,2026-02-20,12,$380,95%,$401.14,$23.90,36.3%,+0.7%,17x,"$2,390",45%,49,67,57
2,AMZN,,⚡ <30 DTE,2026-02-20,12,$205,97%,$210.32,$8.55,36.1%,+1.5%,25x,$855,40%,47,63,54
3,AMZN,,⚡ <30 DTE,2026-02-20,12,$200,95%,$210.32,$12.32,38.3%,+1.0%,17x,"$1,232",44%,46,63,54
4,SNPS,,⚡ <30 DTE,2026-02-20,12,$380,89%,$426.88,$51.10,63.4%,+1.0%,8x,"$5,110",45%,41,61,50
5,SNPS,,⚡ <30 DTE,2026-02-20,12,$400,94%,$426.88,$33.55,54.0%,+1.6%,13x,"$3,355",42%,39,61,49
6,KLAC,,⚡ <30 DTE,2026-02-20,12,"$1,360",94%,"$1,442.95",$103.50,53.1%,+1.4%,14x,"$10,350",43%,62,27,46
7,KLAC,,⚡ <30 DTE,2026-02-20,12,"$1,350",94%,"$1,442.95",$113.50,52.3%,+1.4%,13x,"$11,350",43%,62,27,46
8,UBER,,⚡ <30 DTE,2026-02-20,12,$70,94%,$74.77,$5.40,44.0%,+0.8%,14x,$540,45%,43,49,46
9,ISRG,,⚡ <30 DTE,2026-02-20,12,$430,88%,$488.15,$59.40,64.3%,+0.3%,8x,"$5,940",47%,40,51,45


,,,,+3%,+5%,+8%,+10%
Ticker,Strike,DTE,Cost,,,,
AMAT,300.000000,12,$30.07,+7%,+28%,+61%,+82%
AMZN,205.000000,12,$8.55,+36%,+85%,+159%,+208%
ISRG,430.000000,12,$59.40,+23%,+39%,+64%,+80%
KLAC,1360.000000,12,$103.50,+22%,+50%,+92%,+120%
LRCX,220.000000,19,$19.60,-8%,+15%,+50%,+74%
MSFT,360.000000,12,$42.45,+25%,+44%,+73%,+91%


...

*Deep ITM for delta exposure, ATM for leverage. Long runway = time to be right.*

,Ticker,Conviction,Bucket,Expiry,DTE,Strike,Moneyness,Spot,Mid,IV,Breakeven %,Leverage,Cost/Contract,P(Profit),Edge Score,Bounce Score,Combo Score
0,KLAC,,🏗️ 300+ DTE,2026-09-18,222,"$1,160",80%,"$1,442.95",$189.50,0.0%,-6.5%,8x,"$18,950",100%,86,27,68
1,MSFT,,🏗️ 300+ DTE,2026-06-18,130,$350,87%,$401.14,$64.72,38.5%,+3.4%,6x,"$6,472",42%,55,67,58
2,MSFT,,🏗️ 300+ DTE,2026-05-15,96,$350,87%,$401.14,$62.08,40.4%,+2.7%,6x,"$6,208",43%,54,67,58
3,KLAC,,🏗️ 300+ DTE,2026-06-18,130,"$1,200",83%,"$1,442.95",$325.40,56.8%,+5.7%,4x,"$32,540",39%,67,27,55
4,AMZN,,🏗️ 300+ DTE,2026-06-18,130,$175,83%,$210.32,$43.15,46.1%,+3.7%,5x,"$4,315",42%,47,63,52
5,AMZN,,🏗️ 300+ DTE,2026-06-18,130,$180,86%,$210.32,$39.30,45.1%,+4.3%,5x,"$3,930",41%,47,63,52
6,LRCX,,🏗️ 300+ DTE,2026-06-18,130,$190,82%,$231.01,$59.73,71.4%,+8.1%,4x,"$5,972",36%,67,17,52
7,LRCX,,🏗️ 300+ DTE,2026-06-18,130,$200,87%,$231.01,$53.30,69.9%,+9.6%,4x,"$5,330",35%,66,17,51
8,CMI,,🏗️ 300+ DTE,2026-06-18,130,$480,83%,$577.73,$110.75,38.6%,+2.3%,5x,"$11,075",44%,69,9,51
9,CMI,,🏗️ 300+ DTE,2026-06-18,130,$500,87%,$577.73,$94.95,37.4%,+3.0%,6x,"$9,495",43%,69,9,51


,,,,+10%,+20%,+30%,+50%
Ticker,Strike,DTE,Cost,,,,
AMAT,270.000000,130,$73.75,+15%,+59%,+102%,+190%
AMZN,175.000000,130,$43.15,+31%,+79%,+128%,+226%
CMI,480.000000,130,$110.75,+40%,+93%,+145%,+249%
KLAC,1160.000000,222,$189.50,+125%,+202%,+278%,+430%
LRCX,190.000000,130,$59.73,+7%,+46%,+85%,+162%
MSFT,350.000000,130,$64.72,+41%,+103%,+165%,+289%



---
## 🗒️ Cheat Sheet — Top Pick Per Bucket Per Ticker


,Ticker,Conviction,Bucket,Call,Entry,B/E,Leverage,P(Profit),Edge Score,Bounce Score,Combo Score
0,AMAT,⭐,⚡ <30 DTE,$300 2026-02-20 (12d),$30.07,+2.3%,11x,41%,63,17,42
16,AMAT,⭐,🔄 30-90 DTE,$290 2026-03-20 (40d),$44.77,+3.8%,7x,40%,64,17,40
32,AMAT,⭐,🏗️ 300+ DTE,$270 2026-06-18 (130d),$73.75,+6.6%,4x,38%,65,17,51
1,AMZN,,⚡ <30 DTE,$205 2026-02-20 (12d),$8.55,+1.5%,25x,40%,47,63,54
17,AMZN,,🔄 30-90 DTE,$190 2026-03-20 (40d),$23.77,+1.6%,9x,44%,49,63,56
33,AMZN,,🏗️ 300+ DTE,$175 2026-06-18 (130d),$43.15,+3.7%,5x,42%,47,63,52
2,AVGO,⭐,⚡ <30 DTE,$295 2026-02-20 (12d),$40.83,+0.9%,8x,45%,51,31,42
18,AVGO,⭐,🔄 30-90 DTE,$300 2026-03-20 (40d),$46.62,+4.1%,7x,39%,45,31,38
34,AVGO,⭐,🏗️ 300+ DTE,$280 2026-05-15 (96d),$70.07,+5.2%,5x,39%,49,31,43
3,CMI,,⚡ <30 DTE,$550 2026-02-20 (12d),$30.80,+0.5%,19x,46%,63,9,39


---

## 🏆 Dynamic Pick Engine — Auto-Selects Best Calls

Picks are chosen algorithmically — no hardcoded trades. Re-run any time for fresh results.

### Scoring Factors

| Factor            | What It Measures                                            | Source                        |
| ----------------- | ----------------------------------------------------------- | ----------------------------- |
| **Sharpe Ratio**  | Stock's excess return / volatility (risk-adjusted quality)  | 6M daily returns vs risk-free |
| **Sortino Ratio** | Excess return / target downside deviation (penalises drops) | 6M daily returns (proper TDD) |
| **Calmar Ratio**  | Return / max drawdown (recovery ability)                    | 6M price history              |
| **Win Rate**      | % of positive daily returns (consistency)                   | 6M daily returns              |
| **Edge Score**    | P(Profit), IV vs HV, momentum, breakeven, style, liquidity  | BSM + technicals              |
| **Bounce Score**  | RSI, Stochastic, Bollinger, MACD, support levels            | Technical indicators          |
| **Risk/Reward**   | Expected return at +10% move vs max loss (premium)          | Scenario analysis             |

### Selection Rules per Bucket

- **⚡ <30 DTE**: Top 3 by Final Score (Combo 40%, R/R 25%, Win Rate 20%, Sharpe 15%), max 1 per ticker
- **🔄 30-90 DTE**: Top 3 by Final Score (Combo 35%, Sharpe 20%, Sortino 15%, R/R 15%, Win 15%), max 1 per ticker
- **🏗️ 300+ DTE**: Top 3 by Final Score (Combo 30%, Sortino 20%, Calmar 15%, Sharpe 15%, R/R 10%, Win 10%), max 1 per ticker

### Methodology Notes & Caveats

| Item               | Detail                                                                                                                            |
| ------------------ | --------------------------------------------------------------------------------------------------------------------------------- |
| **P(Profit)**      | Uses BSM risk-neutral probability (drift = r, not μ). Conservative — real-world probability is higher due to equity risk premium. |
| **RSI**            | Wilder's exponential smoothing (EWM com=13), matching industry standard.                                                          |
| **Sortino**        | Target downside deviation: √(mean(min(r − r_f, 0)²)) × √252. Includes zero contributions from up days.                            |
| **IV vs HV**       | Compares option IV to 30-day realised vol. For LEAPS, long-term HV may differ from HV30 (vol term structure).                     |
| **R/R**            | Intrinsic value at expiry only. For LEAPS, early-exit value would include time value → R/R is understated.                        |
| **Annualisation**  | Sharpe/Calmar use arithmetic mean × 252. Geometric (CAGR) would be slightly lower for volatile stocks.                            |
| **Risk-Free Rate** | 4.5% annualised (T-bill proxy), applied consistently to BSM, Sharpe, and Sortino.                                                 |


In [88]:
# ══════════════════════════════════════════════════════════════════
#  🏆  DYNAMIC PICK ENGINE — computes ratios, auto-selects best
# ══════════════════════════════════════════════════════════════════

PICKS_PER_BUCKET = 3

# ──────────────────────────────────────────────────────────────────
# 1.  Compute per-ticker ratios from daily returns (6-month window)
# ──────────────────────────────────────────────────────────────────
print("Computing Sharpe / Sortino / Calmar / Win Rate per ticker …\n")

ticker_ratios = {}
for ticker in ALL_TICKERS:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period="6mo")
        if hist.empty or len(hist) < 60:
            continue
        closes = hist["Close"].dropna()
        daily_ret = closes.pct_change().dropna()

        # Annualised return & vol
        ann_ret = float(daily_ret.mean() * 252)
        ann_vol = float(daily_ret.std() * np.sqrt(252))

        # Sharpe
        sharpe = (ann_ret - RISK_FREE_RATE) / ann_vol if ann_vol > 0 else 0

        # Sortino (target downside deviation — proper formulation)
        daily_rf = RISK_FREE_RATE / 252
        diff_below = np.minimum(daily_ret - daily_rf, 0)
        down_vol = (
            float(np.sqrt(np.mean(diff_below**2)) * np.sqrt(252))
            if len(daily_ret) > 20
            else ann_vol
        )
        sortino = (ann_ret - RISK_FREE_RATE) / down_vol if down_vol > 0 else 0

        # Calmar (return / max drawdown)
        cum = (1 + daily_ret).cumprod()
        peak = cum.cummax()
        drawdown = (cum - peak) / peak
        max_dd = float(drawdown.min())  # negative
        calmar = ann_ret / abs(max_dd) if max_dd != 0 else 0

        # Win rate
        win_rate = float((daily_ret > 0).sum() / len(daily_ret))

        # Beta vs SPY proxy — use simple correlation of returns if we have it
        # (We don't fetch SPY here, so approximate beta from vol ratio)
        beta_approx = ann_vol / 0.16  # assume SPY ~16% vol

        ticker_ratios[ticker] = {
            "ann_ret": ann_ret,
            "ann_vol": ann_vol,
            "sharpe": sharpe,
            "sortino": sortino,
            "calmar": calmar,
            "max_dd": max_dd,
            "win_rate": win_rate,
            "beta": beta_approx,
        }
        flag = "🟢" if sharpe > 0.5 else "🟡" if sharpe > 0 else "🔴"
        print(
            f"  {flag} {ticker:5s}  Sharpe={sharpe:+.2f}  Sortino={sortino:+.2f}  "
            f"Calmar={calmar:+.2f}  MaxDD={max_dd:+.1%}  WinRate={win_rate:.0%}  "
            f"AnnRet={ann_ret:+.1%}  AnnVol={ann_vol:.1%}"
        )
    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")

print(f"\n✅ Ratios computed for {len(ticker_ratios)} tickers")

# ──────────────────────────────────────────────────────────────────
# 2.  Enrich unified_df with ratios + risk/reward
# ──────────────────────────────────────────────────────────────────

# Merge ratios onto unified_df
ratio_df = (
    pd.DataFrame(ticker_ratios).T.reset_index().rename(columns={"index": "Ticker"})
)
ratio_df = ratio_df.rename(
    columns={
        "sharpe": "Sharpe",
        "sortino": "Sortino",
        "calmar": "Calmar",
        "max_dd": "Max DD",
        "win_rate": "Win Rate",
        "ann_ret": "Ann Ret",
        "ann_vol": "Ann Vol",
        "beta": "Beta",
    }
)

# Drop old ratio columns if re-running
for c in [
    "Sharpe",
    "Sortino",
    "Calmar",
    "Max DD",
    "Win Rate",
    "Ann Ret",
    "Ann Vol",
    "Beta",
    "R/R",
    "Final Score",
]:
    if c in unified_df.columns:
        unified_df = unified_df.drop(columns=[c])

unified_df = unified_df.merge(
    ratio_df[
        [
            "Ticker",
            "Sharpe",
            "Sortino",
            "Calmar",
            "Max DD",
            "Win Rate",
            "Ann Ret",
            "Ann Vol",
            "Beta",
        ]
    ],
    on="Ticker",
    how="left",
)


# Risk/Reward: expected return at +10% stock move vs max loss (premium)
def calc_rr(row):
    spot = row["Spot"]
    strike = row["Strike"]
    mid = row["Mid"]
    if mid <= 0 or spot <= 0:
        return np.nan
    upside_spot = spot * 1.10
    payout = max(upside_spot - strike, 0)
    gain = (payout - mid) / mid  # % return if stock +10%
    loss = -1.0  # max loss = 100% of premium
    return gain / abs(loss) if loss != 0 else gain  # reward-to-risk


unified_df["R/R"] = unified_df.apply(calc_rr, axis=1)

# ──────────────────────────────────────────────────────────────────
# 3.  Final Score — bucket-specific weighting
# ──────────────────────────────────────────────────────────────────


def final_score(row):
    combo = row["Combo Score"] if not np.isnan(row["Combo Score"]) else 0
    sharpe = row.get("Sharpe", 0) if not np.isnan(row.get("Sharpe", np.nan)) else 0
    sortino = row.get("Sortino", 0) if not np.isnan(row.get("Sortino", np.nan)) else 0
    rr = row.get("R/R", 0) if not np.isnan(row.get("R/R", np.nan)) else 0
    win = row.get("Win Rate", 0.5) if not np.isnan(row.get("Win Rate", np.nan)) else 0.5
    calmar = row.get("Calmar", 0) if not np.isnan(row.get("Calmar", np.nan)) else 0

    # Normalize ratios to ~0-100 scale
    sharpe_n = np.clip((sharpe + 1) * 33, 0, 100)  # -1→0, 0→33, 2→100
    sortino_n = np.clip((sortino + 1) * 25, 0, 100)  # wider range
    rr_n = np.clip(rr * 20, 0, 100)  # 0→0, 5→100
    win_n = np.clip(win * 100 - 20, 0, 100)  # 50%→30, 60%→40
    calmar_n = np.clip(calmar * 25, 0, 100)  # 0→0, 4→100

    bucket = row["Bucket"]
    if bucket == "⚡ <30 DTE":
        # Short-term: care about momentum (Combo), risk/reward, win rate
        return combo * 0.40 + rr_n * 0.25 + sharpe_n * 0.15 + win_n * 0.20
    elif bucket == "🔄 30-90 DTE":
        # Medium: balanced — sharpe + combo + sortino
        return (
            combo * 0.35
            + sharpe_n * 0.20
            + sortino_n * 0.15
            + rr_n * 0.15
            + win_n * 0.15
        )
    else:
        # LEAPS: quality matters most — sortino, calmar, combo
        return (
            combo * 0.30
            + sortino_n * 0.20
            + calmar_n * 0.15
            + sharpe_n * 0.15
            + rr_n * 0.10
            + win_n * 0.10
        )


unified_df["Final Score"] = unified_df.apply(final_score, axis=1)

# ──────────────────────────────────────────────────────────────────
# 4.  Auto-select picks per bucket
# ──────────────────────────────────────────────────────────────────

display(Markdown("---\n# 🏆 Auto-Selected Picks\n"))

bucket_configs = [
    (
        "⚡ <30 DTE",
        "⚡ Short-Term Swings",
        "Quick directional bets — high leverage, tight risk. Prioritises R/R asymmetry + win rate.",
    ),
    (
        "🔄 30-90 DTE",
        "🔄 Medium-Term Bounce & Recovery",
        "1-3 month horizon for oversold reversals. Sharpe & Sortino weigh quality of the underlying move.",
    ),
    (
        "🏗️ 300+ DTE",
        "🏗️ LEAPS — Conviction Holds",
        "Stock replacement & deep-ITM compounding. Sortino + Calmar reward consistent upside with limited drawdowns.",
    ),
]

all_auto_picks = []

for bucket_key, title, desc in bucket_configs:
    sub = unified_df[unified_df["Bucket"] == bucket_key].copy()
    if sub.empty:
        display(Markdown(f"\n### {title}\n*No options in this bucket.*"))
        continue

    # Pick best 1 per ticker, then top N
    top = (
        sub.sort_values("Final Score", ascending=False)
        .groupby("Ticker")
        .head(1)
        .sort_values("Final Score", ascending=False)
        .head(PICKS_PER_BUCKET)
        .reset_index(drop=True)
    )

    display(Markdown(f"\n### {title}"))
    display(Markdown(f"*{desc}*"))

    # Detailed card per pick
    for rank, (_, r) in enumerate(top.iterrows(), 1):
        pp_str = f"{r['P(Profit)']:.0%}" if not np.isnan(r["P(Profit)"]) else "—"
        be_str = f"{r['Breakeven %']:+.1%}" if not np.isnan(r["Breakeven %"]) else "—"
        sharpe_val = r.get("Sharpe", np.nan)
        sortino_val = r.get("Sortino", np.nan)
        calmar_val = r.get("Calmar", np.nan)
        maxdd_val = r.get("Max DD", np.nan)
        winr_val = r.get("Win Rate", np.nan)
        rr_val = r.get("R/R", np.nan)
        bounce_val = r.get("Bounce Score", 0)
        conv = "⭐ " if r["Ticker"] in CONVICTION else ""

        # Auto-generate thesis based on data
        reasons = []
        if not np.isnan(sharpe_val) and sharpe_val > 0.5:
            reasons.append(
                f"Strong Sharpe ({sharpe_val:+.2f}) — quality risk-adjusted returns"
            )
        elif not np.isnan(sharpe_val) and sharpe_val < -0.5:
            reasons.append(
                f"Negative Sharpe ({sharpe_val:+.2f}) — contrarian bounce play"
            )
        if not np.isnan(sortino_val) and sortino_val > 0.5:
            reasons.append(
                f"Sortino {sortino_val:+.2f} — upside moves dominate downside"
            )
        if not np.isnan(calmar_val) and calmar_val > 1.0:
            reasons.append(f"Calmar {calmar_val:.1f} — strong recovery from drawdowns")
        if not np.isnan(maxdd_val) and maxdd_val > -0.15:
            reasons.append(f"Shallow max drawdown ({maxdd_val:.0%})")
        if not np.isnan(winr_val) and winr_val > 0.53:
            reasons.append(f"Win rate {winr_val:.0%} — more up days than down")
        if bounce_val >= 50:
            reasons.append(
                f"Bounce Score {bounce_val:.0f} — multiple oversold indicators firing"
            )
        elif bounce_val >= 30:
            reasons.append(
                f"Bounce Score {bounce_val:.0f} — developing reversal signals"
            )
        if not np.isnan(rr_val) and rr_val > 2:
            reasons.append(f"R/R = {rr_val:.1f}x at +10% stock move")
        if r["Moneyness"] < 0.88:
            reasons.append(
                f"Deep ITM ({r['Moneyness']:.0%}) — high delta, stock replacement"
            )
        if not np.isnan(r.get("RSI", np.nan)) and r["RSI"] < 30:
            reasons.append(f"RSI {r['RSI']:.0f} — oversold")
        if r["Ticker"] in CONVICTION:
            reasons.append("Conviction name ⭐")

        why_text = (
            ". ".join(reasons) if reasons else "Scored highest in composite ranking."
        )

        # Risk auto-generated
        risks = []
        if r["DTE"] < 20:
            risks.append("Very short runway — needs quick catalyst")
        if not np.isnan(r.get("Ret 1M", np.nan)) and r["Ret 1M"] < -0.10:
            risks.append(f"Down {r['Ret 1M']:.0%} in last month — could keep falling")
        if not np.isnan(maxdd_val) and maxdd_val < -0.25:
            risks.append(f"Deep drawdown history ({maxdd_val:.0%})")
        if not np.isnan(r["IV"]) and r["IV"] > 0.50:
            risks.append(f"Elevated IV ({r['IV']:.0%}) — paying up for premium")
        if r.get("OI", 0) < 50:
            risks.append("Low open interest — wide spreads likely")
        risk_text = ". ".join(risks) if risks else "Standard directional risk."

        # Sizing logic
        if r["DTE"] < 30:
            sizing = "Small (1-2%)"
        elif r["DTE"] <= 90:
            sizing = "Medium (2-4%)"
        elif r["Moneyness"] < 0.85:
            sizing = "Large (5-8%) — stock replacement"
        else:
            sizing = "Medium (2-4%)"

        display(
            Markdown(
                f"#### #{rank} {conv}{r['Ticker']} — ${r['Strike']:,.0f} {r['Expiry']}  ({r['DTE']}d)\n"
                f"| Metric | Value | | Metric | Value |\n"
                f"|--------|-------|-|--------|-------|\n"
                f"| Entry | **${r['Mid']:,.2f}** (${r['Mid'] * 100:,.0f}/ct) | "
                f"| Sharpe | {sharpe_val:+.2f} |\n"
                f"| Breakeven | {be_str} from spot | "
                f"| Sortino | {sortino_val:+.2f} |\n"
                f"| Leverage | {r['Leverage']:.0f}x | "
                f"| Calmar | {calmar_val:+.2f} |\n"
                f"| P(Profit) | {pp_str} | "
                f"| Max DD | {maxdd_val:.0%} |\n"
                f"| R/R @+10% | {rr_val:.1f}x | "
                f"| Win Rate | {winr_val:.0%} |\n"
                f"| Edge / Bounce | {r['Edge Score']:.0f} / {bounce_val:.0f} | "
                f"| **Final Score** | **{r['Final Score']:.0f}** |\n"
                f"| Sizing | {sizing} | | | |\n\n"
                f"**Why:** {why_text}\n\n"
                f"**Risk:** {risk_text}"
            )
        )

    all_auto_picks.append(top)

# ══════════════════════════════════════════════════════════════════
# Summary table — all picks at a glance
# ══════════════════════════════════════════════════════════════════

display(Markdown("\n---\n### 📋 All Picks at a Glance\n"))

if all_auto_picks:
    summary = pd.concat(all_auto_picks, ignore_index=True)

    sum_cols = [
        "Bucket",
        "Ticker",
        "Conviction",
        "Expiry",
        "DTE",
        "Strike",
        "Moneyness",
        "Mid",
        "Breakeven %",
        "Leverage",
        "P(Profit)",
        "Sharpe",
        "Sortino",
        "Calmar",
        "Max DD",
        "Win Rate",
        "R/R",
        "Edge Score",
        "Bounce Score",
        "Final Score",
    ]
    sa = [c for c in sum_cols if c in summary.columns]

    display(
        summary[sa]
        .style.format(
            {
                "Strike": "${:,.0f}",
                "Mid": "${:,.2f}",
                "Moneyness": "{:.0%}",
                "Breakeven %": "{:+.1%}",
                "Leverage": "{:.0f}x",
                "P(Profit)": "{:.0%}",
                "Sharpe": "{:+.2f}",
                "Sortino": "{:+.2f}",
                "Calmar": "{:+.2f}",
                "Max DD": "{:.0%}",
                "Win Rate": "{:.0%}",
                "R/R": "{:.1f}x",
                "Edge Score": "{:.0f}",
                "Bounce Score": "{:.0f}",
                "Final Score": "{:.0f}",
            },
            na_rep="—",
        )
        .background_gradient(subset=["Final Score"], cmap="RdYlGn", vmin=30, vmax=65)
        .background_gradient(subset=["Sharpe"], cmap="RdYlGn", vmin=-1, vmax=2)
        .background_gradient(subset=["R/R"], cmap="Greens", vmin=0, vmax=5)
        .set_caption("Auto-selected picks — ranked by Final Score per bucket")
    )

# ══════════════════════════════════════════════════════════════════
# Scenario P&L for all auto-picks
# ══════════════════════════════════════════════════════════════════

display(Markdown("\n### 💰 Return Scenarios — All Picks\n"))

if all_auto_picks:
    scen_all = []
    for _, r in summary.iterrows():
        for m in [-0.05, 0.05, 0.10, 0.15, 0.25]:
            new_spot = r["Spot"] * (1 + m)
            payout = max(new_spot - r["Strike"], 0)
            ret = (payout - r["Mid"]) / r["Mid"] if r["Mid"] > 0 else 0
            scen_all.append(
                {
                    "Bucket": r["Bucket"],
                    "Ticker": r["Ticker"],
                    "Strike": f"${r['Strike']:,.0f}",
                    "DTE": int(r["DTE"]),
                    "Cost": f"${r['Mid']:,.2f}",
                    "Move": m,
                    "Return": ret,
                }
            )
    scen_all_df = pd.DataFrame(scen_all)
    piv = scen_all_df.pivot_table(
        index=["Bucket", "Ticker", "Strike", "DTE", "Cost"],
        columns="Move",
        values="Return",
    )
    piv.columns = [f"{c:+.0%}" for c in piv.columns]
    display(
        piv.style.format("{:+.0%}", na_rep="—")
        .background_gradient(cmap="RdYlGn", vmin=-1.0, vmax=3.0, axis=None)
        .set_caption("Return at expiry if stock moves — all auto-picks")
    )

# ══════════════════════════════════════════════════════════════════
# Ratio comparison chart
# ══════════════════════════════════════════════════════════════════

display(Markdown("\n### 📊 Ratio Dashboard — Picked Tickers\n"))

if all_auto_picks:
    picked_tickers = summary["Ticker"].unique().tolist()
    ratio_plot = ratio_df[ratio_df["Ticker"].isin(picked_tickers)].copy()

    fig_ratios = go.Figure()
    fig_ratios.add_trace(
        go.Bar(
            name="Sharpe",
            x=ratio_plot["Ticker"],
            y=ratio_plot["Sharpe"],
            marker_color="#3498db",
        )
    )
    fig_ratios.add_trace(
        go.Bar(
            name="Sortino",
            x=ratio_plot["Ticker"],
            y=ratio_plot["Sortino"],
            marker_color="#2ecc71",
        )
    )
    fig_ratios.add_trace(
        go.Bar(
            name="Calmar",
            x=ratio_plot["Ticker"],
            y=ratio_plot["Calmar"],
            marker_color="#e67e22",
        )
    )
    fig_ratios.update_layout(
        barmode="group",
        title="Risk-Adjusted Ratios — Picked Tickers",
        yaxis_title="Ratio Value",
        height=400,
    )
    fig_ratios.add_hline(y=0, line_dash="dot", line_color="gray")
    fig_ratios.show()

    # Scatter: Final Score vs Sharpe
    fig_fs = px.scatter(
        summary,
        x="Sharpe",
        y="Final Score",
        text="Ticker",
        color="Bucket",
        size="Leverage",
        size_max=25,
        color_discrete_map={
            "⚡ <30 DTE": "#e74c3c",
            "🔄 30-90 DTE": "#f39c12",
            "🏗️ 300+ DTE": "#2ecc71",
        },
        title="Final Score vs Sharpe Ratio (size = leverage)",
    )
    fig_fs.update_traces(textposition="top center")
    fig_fs.show()

Computing Sharpe / Sortino / Calmar / Win Rate per ticker …

  🟢 WDC    Sharpe=+4.13  Sortino=+7.80  Calmar=+14.53  MaxDD=-20.1%  WinRate=62%  AnnRet=+292.2%  AnnVol=69.7%
  🟢 GEV    Sharpe=+0.91  Sortino=+1.45  Calmar=+3.03  MaxDD=-16.6%  WinRate=48%  AnnRet=+50.3%  AnnVol=50.4%
  🟢 STX    Sharpe=+3.34  Sortino=+6.39  Calmar=+12.40  MaxDD=-19.2%  WinRate=59%  AnnRet=+238.3%  AnnVol=69.9%
  🟢 LRCX   Sharpe=+3.43  Sortino=+5.56  Calmar=+11.43  MaxDD=-16.1%  WinRate=63%  AnnRet=+184.0%  AnnVol=52.3%
  🟢 AMAT   Sharpe=+2.55  Sortino=+3.75  Calmar=+7.17  MaxDD=-17.5%  WinRate=60%  AnnRet=+125.8%  AnnVol=47.6%
  🟢 TSM    Sharpe=+2.20  Sortino=+3.55  Calmar=+7.62  MaxDD=-10.5%  WinRate=54%  AnnRet=+79.7%  AnnVol=34.2%
  🟢 GE     Sharpe=+1.19  Sortino=+1.70  Calmar=+3.50  MaxDD=-10.7%  WinRate=57%  AnnRet=+37.5%  AnnVol=27.8%
  🟢 CMI    Sharpe=+2.62  Sortino=+4.03  Calmar=+8.25  MaxDD=-10.7%  WinRate=58%  AnnRet=+88.5%  AnnVol=32.1%
  🟢 KLAC   Sharpe=+2.04  Sortino=+2.90  Calmar=+4.68  MaxDD=

---
# 🏆 Auto-Selected Picks



### ⚡ Short-Term Swings

*Quick directional bets — high leverage, tight risk. Prioritises R/R asymmetry + win rate.*

#### #1 CMI — $590 2026-02-20  (12d)
| Metric | Value | | Metric | Value |
|--------|-------|-|--------|-------|
| Entry | **$5.90** ($590/ct) | | Sharpe | +2.62 |
| Breakeven | +3.1% from spot | | Sortino | +4.03 |
| Leverage | 98x | | Calmar | +8.25 |
| P(Profit) | 27% | | Max DD | -11% |
| R/R @+10% | 6.7x | | Win Rate | 58% |
| Edge / Bounce | 49 / 9 | | **Final Score** | **60** |
| Sizing | Small (1-2%) | | | |

**Why:** Strong Sharpe (+2.62) — quality risk-adjusted returns. Sortino +4.03 — upside moves dominate downside. Calmar 8.3 — strong recovery from drawdowns. Shallow max drawdown (-11%). Win rate 58% — more up days than down. R/R = 6.7x at +10% stock move

**Risk:** Very short runway — needs quick catalyst

...

#### #3 CMI — $480 2026-06-18  (130d)
| Metric | Value | | Metric | Value |
|--------|-------|-|--------|-------|
| Entry | **$110.75** ($11,075/ct) | | Sharpe | +2.62 |
| Breakeven | +2.3% from spot | | Sortino | +4.03 |
| Leverage | 5x | | Calmar | +8.25 |
| P(Profit) | 44% | | Max DD | -11% |
| R/R @+10% | 0.4x | | Win Rate | 58% |
| Edge / Bounce | 69 / 9 | | **Final Score** | **70** |
| Sizing | Large (5-8%) — stock replacement | | | |

**Why:** Strong Sharpe (+2.62) — quality risk-adjusted returns. Sortino +4.03 — upside moves dominate downside. Calmar 8.3 — strong recovery from drawdowns. Shallow max drawdown (-11%). Win rate 58% — more up days than down. Deep ITM (83%) — high delta, stock replacement

**Risk:** Low open interest — wide spreads likely


---
### 📋 All Picks at a Glance


,Bucket,Ticker,Conviction,Expiry,DTE,Strike,Moneyness,Mid,Breakeven %,Leverage,P(Profit),Sharpe,Sortino,Calmar,Max DD,Win Rate,R/R,Edge Score,Bounce Score,Final Score
0,⚡ <30 DTE,CMI,,2026-02-20,12,$590,102%,$5.90,+3.1%,98x,27%,+2.62,+4.03,+8.25,-11%,58%,6.7x,49,9,60
1,⚡ <30 DTE,GE,,2026-02-20,12,$330,103%,$3.85,+4.0%,83x,25%,+1.19,+1.70,+3.50,-11%,57%,5.0x,35,15,54
2,⚡ <30 DTE,AMZN,,2026-02-20,12,$220,105%,$1.92,+5.5%,110x,19%,-0.37,-0.53,-0.41,-17%,54%,4.9x,33,63,53
3,🔄 30-90 DTE,KLAC,,2026-03-20,40,"$1,280",89%,$201.75,+2.7%,7x,42%,+2.04,+2.90,+4.68,-22%,60%,0.5x,69,27,59
4,🔄 30-90 DTE,LRCX,,2026-03-20,40,$210,91%,$30.75,+4.2%,8x,39%,+3.43,+5.56,+11.43,-16%,63%,0.4x,67,17,58
5,🔄 30-90 DTE,CMI,,2026-03-20,40,$520,90%,$62.75,+0.9%,9x,46%,+2.62,+4.03,+8.25,-11%,58%,0.8x,69,9,57
6,🏗️ 300+ DTE,KLAC,,2026-09-18,222,"$1,160",80%,$189.50,-6.5%,8x,100%,+2.04,+2.90,+4.68,-22%,60%,1.3x,86,27,76
7,🏗️ 300+ DTE,LRCX,,2026-06-18,130,$190,82%,$59.73,+8.1%,4x,36%,+3.43,+5.56,+11.43,-16%,63%,0.1x,67,17,70
8,🏗️ 300+ DTE,CMI,,2026-06-18,130,$480,83%,$110.75,+2.3%,5x,44%,+2.62,+4.03,+8.25,-11%,58%,0.4x,69,9,70



### 💰 Return Scenarios — All Picks



### 📊 Ratio Dashboard — Picked Tickers


---

## 💼 Portfolio Construction Engine — $15k Starting Balance

Builds an optimal, diversified portfolio of call options across tickers and time horizons.

### How It Works

1. **Fetch fundamentals** — Forward P/E, PEG, FCF yield, market cap via `yfinance .info`. Compute a Valuation Score (0–100) rewarding cheap + growing names.
2. **Correlation matrix** — 6-month daily returns across all tickers. Used to penalise stacking correlated positions (e.g. LRCX + AMAT).
3. **Portfolio Score** — Extends Final Score with valuation quality + diversification benefit.
4. **Fractional Kelly sizing** — `f* = (p·b − q) / b` capped at 25% Kelly, max 12% of portfolio per position. Converts to real dollar allocations.
5. **Greedy builder** — Iteratively selects highest Portfolio Score option that fits remaining balance, enforcing:
   - Max ~20% of total capital per single ticker (across all buckets)
   - Target bucket split: ~15–20% short / ~25–30% medium / ~50–60% LEAPS
   - Correlation penalty: score reduced if ρ > 0.65 with an already-held name
   - Whole-contract constraint (options = 100 shares, can't buy fractional)
6. **Dashboard** — Allocation charts, risk summary, scenario P&L at portfolio level, correlation heatmap.


In [ ]:
# ══════════════════════════════════════════════════════════════════
#  💼  PORTFOLIO CONSTRUCTION ENGINE
# ══════════════════════════════════════════════════════════════════

STARTING_BALANCE = 15_000
MAX_TICKER_PCT = 0.25  # max 25% of capital in any one ticker
MAX_SINGLE_POS_PCT = 0.20  # max 20% per single option position
FRACTIONAL_KELLY = 0.25  # use 25% of full Kelly (conservative)
KELLY_CAP = 0.20  # hard cap Kelly fraction (higher for small accounts)
EDGE_HAIRCUT = 0.90  # haircut BSM P(Profit) by 10% for conservatism
CORRELATION_PENALTY_THRESH = 0.65  # penalise if ρ > this with existing position
MIN_POSITION_COST = 100  # don't bother with positions under $100

# Target bucket allocation (soft targets — greedy builder aims for these)
BUCKET_TARGET = {
    "⚡ <30 DTE": 0.15,  # 15% short-term
    "🔄 30-90 DTE": 0.30,  # 30% medium-term
    "🏗️ 300+ DTE": 0.55,  # 55% LEAPS
}

# ──────────────────────────────────────────────────────────────────
# 1.  FETCH FUNDAMENTALS — valuation data from yfinance .info
# ──────────────────────────────────────────────────────────────────
print("Fetching fundamentals (P/E, PEG, FCF, market cap)…\n")

fundamentals = {}
for ticker in ALL_TICKERS:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        info = t.info or {}

        fwd_pe = info.get("forwardPE") or info.get("forwardPe")
        trail_pe = info.get("trailingPE") or info.get("trailingPe")
        peg = info.get("pegRatio") or info.get("trailingPegRatio")
        mcap = info.get("marketCap", 0)
        fcf = info.get("freeCashflow", 0)
        ev = info.get("enterpriseValue", 0)
        rev_growth = info.get("revenueGrowth")
        profit_margin = info.get("profitMargins")
        roe = info.get("returnOnEquity")

        # FCF yield = FCF / Market Cap
        fcf_yield = (fcf / mcap * 100) if (mcap and fcf) else None

        fundamentals[ticker] = {
            "fwd_pe": fwd_pe,
            "trail_pe": trail_pe,
            "peg": peg,
            "mcap": mcap,
            "fcf_yield": fcf_yield,
            "rev_growth": rev_growth,
            "profit_margin": profit_margin,
            "roe": roe,
        }

        pe_str = f"FwdPE={fwd_pe:.1f}" if fwd_pe else "FwdPE=—"
        peg_str = f"PEG={peg:.2f}" if peg else "PEG=—"
        fcf_str = f"FCFy={fcf_yield:.1f}%" if fcf_yield else "FCFy=—"
        mcap_str = f"MCap=${mcap / 1e9:.0f}B" if mcap else "MCap=—"
        print(f"  {ticker:5s}  {pe_str:14s}  {peg_str:10s}  {fcf_str:10s}  {mcap_str}")
    except Exception as e:
        print(f"  ⚠ {ticker}: {e}")

print(f"\n✅ Fundamentals for {len(fundamentals)} tickers")


# ──────────────────────────────────────────────────────────────────
# 2.  VALUATION SCORE (0–100)
# ──────────────────────────────────────────────────────────────────


def valuation_score(ticker):
    """Score 0-100: higher = more attractively valued."""
    f = fundamentals.get(ticker, {})
    if not f:
        return 50  # neutral default

    score = 0
    n = 0

    # Forward P/E — lower is better for value, but not too low (value trap)
    fwd_pe = f.get("fwd_pe")
    if fwd_pe and fwd_pe > 0:
        if fwd_pe < 12:
            score += 85
        elif fwd_pe < 18:
            score += 75
        elif fwd_pe < 25:
            score += 60
        elif fwd_pe < 35:
            score += 40
        elif fwd_pe < 50:
            score += 25
        else:
            score += 10
        n += 1

    # PEG ratio — <1 is ideal (cheap relative to growth)
    peg = f.get("peg")
    if peg and peg > 0:
        if peg < 0.8:
            score += 90
        elif peg < 1.2:
            score += 75
        elif peg < 2.0:
            score += 50
        elif peg < 3.0:
            score += 30
        else:
            score += 10
        n += 1

    # FCF yield — higher is better (cash generation)
    fcf_y = f.get("fcf_yield")
    if fcf_y is not None:
        if fcf_y > 6:
            score += 90
        elif fcf_y > 4:
            score += 75
        elif fcf_y > 2:
            score += 55
        elif fcf_y > 0:
            score += 35
        else:
            score += 10  # negative FCF
        n += 1

    # Revenue growth — bonus for growing companies
    rg = f.get("rev_growth")
    if rg is not None:
        if rg > 0.20:
            score += 85
        elif rg > 0.10:
            score += 65
        elif rg > 0.05:
            score += 50
        elif rg > 0:
            score += 35
        else:
            score += 15
        n += 1

    # Profit margin — quality indicator
    pm = f.get("profit_margin")
    if pm is not None:
        if pm > 0.30:
            score += 85
        elif pm > 0.20:
            score += 70
        elif pm > 0.10:
            score += 50
        elif pm > 0:
            score += 30
        else:
            score += 10
        n += 1

    return score / n if n > 0 else 50


val_scores = {t: valuation_score(t) for t in ALL_TICKERS}
val_df = pd.DataFrame(
    [
        {
            "Ticker": t,
            "Val Score": s,
            "Fwd PE": fundamentals.get(t, {}).get("fwd_pe"),
            "PEG": fundamentals.get(t, {}).get("peg"),
            "FCF Yield": fundamentals.get(t, {}).get("fcf_yield"),
            "Rev Growth": fundamentals.get(t, {}).get("rev_growth"),
            "Profit Margin": fundamentals.get(t, {}).get("profit_margin"),
        }
        for t, s in val_scores.items()
    ]
).sort_values("Val Score", ascending=False)

display(Markdown("### 📊 Valuation Scores"))
display(
    val_df.style.format(
        {
            "Val Score": "{:.0f}",
            "Fwd PE": "{:.1f}",
            "PEG": "{:.2f}",
            "FCF Yield": "{:.1f}%",
            "Rev Growth": "{:.1%}",
            "Profit Margin": "{:.1%}",
        },
        na_rep="—",
    )
    .background_gradient(subset=["Val Score"], cmap="RdYlGn", vmin=30, vmax=80)
    .set_caption("Higher = more attractively valued")
)


# ──────────────────────────────────────────────────────────────────
# 3.  CORRELATION MATRIX — 6-month daily returns
# ──────────────────────────────────────────────────────────────────
print("\nBuilding correlation matrix…")

returns_dict = {}
for ticker in ALL_TICKERS:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period="6mo")
        if not hist.empty and len(hist) > 30:
            returns_dict[ticker] = hist["Close"].pct_change().dropna()
    except Exception:
        pass

returns_matrix = pd.DataFrame(returns_dict)
corr_matrix = returns_matrix.corr()
print(f"✅ Correlation matrix: {corr_matrix.shape[0]}×{corr_matrix.shape[1]}")


# ──────────────────────────────────────────────────────────────────
# 4.  PORTFOLIO SCORE — Final Score + Valuation + Diversification
# ──────────────────────────────────────────────────────────────────

# Merge valuation scores onto unified_df
unified_df["Val Score"] = unified_df["Ticker"].map(val_scores).fillna(50)


def portfolio_score(row):
    """Extend Final Score with valuation quality."""
    fs = row.get("Final Score", 0)
    if np.isnan(fs):
        fs = 0
    vs = row.get("Val Score", 50)

    bucket = row["Bucket"]
    if bucket == "⚡ <30 DTE":
        # Short-term: valuation matters less, momentum matters more
        return fs * 0.85 + vs * 0.15
    elif bucket == "🔄 30-90 DTE":
        # Medium: valuation starts to matter
        return fs * 0.80 + vs * 0.20
    else:
        # LEAPS: valuation matters most — you're a long-term holder
        return fs * 0.70 + vs * 0.30


unified_df["Portfolio Score"] = unified_df.apply(portfolio_score, axis=1)


# ──────────────────────────────────────────────────────────────────
# 5.  FRACTIONAL KELLY SIZING
# ──────────────────────────────────────────────────────────────────


def kelly_fraction(p_profit, rr):
    """Fractional Kelly criterion for position sizing.

    f* = (p*b - q) / b   where p=P(win), q=1-p, b=win/loss ratio
    Then multiply by FRACTIONAL_KELLY for conservatism.
    """
    if np.isnan(p_profit) or np.isnan(rr) or rr <= 0:
        return 0.01  # minimum 1% if we can't compute

    p = min(p_profit * EDGE_HAIRCUT, 0.95)  # haircut + cap
    q = 1 - p
    b = rr  # reward-to-risk ratio

    full_kelly = (p * b - q) / b if b > 0 else 0

    if full_kelly <= 0:
        return 0.005  # tiny allocation for negative-edge (included for diversification)

    frac = full_kelly * FRACTIONAL_KELLY
    return min(frac, KELLY_CAP)


def max_contracts(kelly_frac, balance, cost_per_contract):
    """How many whole contracts can we buy given Kelly fraction and balance."""
    if cost_per_contract <= 0:
        return 0
    dollar_alloc = balance * kelly_frac
    return max(0, int(dollar_alloc // cost_per_contract))


# ──────────────────────────────────────────────────────────────────
# 6.  ROUND-ROBIN PORTFOLIO BUILDER — fills buckets proportionally
# ──────────────────────────────────────────────────────────────────
print("\n" + "═" * 60)
print("  💼  BUILDING PORTFOLIO — Round-Robin Bucket Builder")
print("═" * 60)
print(f"  Starting balance: ${STARTING_BALANCE:,.0f}")
print(f"  Kelly fraction:   {FRACTIONAL_KELLY:.0%} of full Kelly")
print(f"  Edge haircut:     {EDGE_HAIRCUT:.0%} of BSM P(Profit)")
print(f"  Max per ticker:   {MAX_TICKER_PCT:.0%}")
print(f"  Max per position: {MAX_SINGLE_POS_PCT:.0%}")
print(
    f"  Bucket targets:   LEAPS {BUCKET_TARGET['🏗️ 300+ DTE']:.0%} / "
    f"Med {BUCKET_TARGET['🔄 30-90 DTE']:.0%} / "
    f"Short {BUCKET_TARGET['⚡ <30 DTE']:.0%}\n"
)

# Pre-compute Kelly fractions for all candidates
candidates = unified_df.copy()
candidates["Kelly Frac"] = candidates.apply(
    lambda r: kelly_fraction(r.get("P(Profit)", np.nan), r.get("R/R", np.nan)),
    axis=1,
)

# Filter: must have valid cost/score, and at least 1 contract affordable
candidates = candidates[
    (candidates["Cost/Contract"] > 0)
    & (candidates["Portfolio Score"] > 0)
    & (candidates["Cost/Contract"] <= STARTING_BALANCE)
].copy()

# Track state
remaining = STARTING_BALANCE
ticker_allocated = {}  # ticker → total $ allocated
bucket_allocated = {}  # bucket → total $ allocated
selected_tickers = set()  # for correlation penalty
selected_options = set()  # (ticker, strike, expiry) to avoid duplicates
portfolio_rows = []

# Bucket dollar targets
bucket_targets = {b: STARTING_BALANCE * pct for b, pct in BUCKET_TARGET.items()}

# Build candidate pools per bucket, sorted by Portfolio Score
bucket_order = ["🏗️ 300+ DTE", "🔄 30-90 DTE", "⚡ <30 DTE"]  # LEAPS first
bucket_pools = {}
for b in bucket_order:
    pool = candidates[candidates["Bucket"] == b].sort_values(
        "Portfolio Score", ascending=False
    )
    bucket_pools[b] = pool.iterrows()


def try_add_position(
    row,
    remaining,
    ticker_allocated,
    bucket_allocated,
    selected_tickers,
    selected_options,
    portfolio_rows,
):
    """Try to add a single position. Returns (remaining, success)."""
    ticker = row["Ticker"]
    bucket = row["Bucket"]
    cost = row["Cost/Contract"]
    opt_key = (ticker, row["Strike"], row["Expiry"])

    if opt_key in selected_options:
        return remaining, False
    if cost > remaining:
        return remaining, False

    # Ticker concentration
    ticker_total = ticker_allocated.get(ticker, 0)
    if ticker_total >= STARTING_BALANCE * MAX_TICKER_PCT:
        return remaining, False

    # Correlation penalty
    corr_penalty = 1.0
    for held_ticker in selected_tickers:
        if held_ticker in corr_matrix.columns and ticker in corr_matrix.columns:
            rho = corr_matrix.loc[ticker, held_ticker]
            if rho > CORRELATION_PENALTY_THRESH:
                corr_penalty = min(
                    corr_penalty, 1.0 - (rho - CORRELATION_PENALTY_THRESH) * 1.5
                )

    effective_score = row["Portfolio Score"] * max(corr_penalty, 0.3)
    if effective_score < 20:
        return remaining, False

    # Position size
    kelly_f = row["Kelly Frac"]
    min_alloc = min(cost, STARTING_BALANCE * 0.05)
    ticker_room = STARTING_BALANCE * MAX_TICKER_PCT - ticker_total

    dollar_alloc = min(
        remaining,
        max(STARTING_BALANCE * kelly_f, min_alloc),
        STARTING_BALANCE * MAX_SINGLE_POS_PCT,
        ticker_room,
    )

    n_contracts = int(dollar_alloc // cost)
    if n_contracts < 1:
        # Try to squeeze in 1 contract if affordable
        if cost <= remaining and cost <= ticker_room:
            n_contracts = 1
        else:
            return remaining, False

    total_cost = n_contracts * cost
    if total_cost > remaining:
        n_contracts = int(remaining // cost)
        if n_contracts < 1:
            return remaining, False
        total_cost = n_contracts * cost

    # Record
    remaining -= total_cost
    ticker_allocated[ticker] = ticker_allocated.get(ticker, 0) + total_cost
    bucket_allocated[bucket] = bucket_allocated.get(bucket, 0) + total_cost
    selected_tickers.add(ticker)
    selected_options.add(opt_key)

    portfolio_rows.append(
        {
            "Ticker": ticker,
            "Conviction": row.get("Conviction", ""),
            "Bucket": bucket,
            "Expiry": row["Expiry"],
            "DTE": row["DTE"],
            "Strike": row["Strike"],
            "Moneyness": row["Moneyness"],
            "Spot": row["Spot"],
            "Mid": row["Mid"],
            "Contracts": n_contracts,
            "Total Cost": total_cost,
            "% of Portfolio": total_cost / STARTING_BALANCE,
            "Kelly Frac": kelly_f,
            "P(Profit)": row.get("P(Profit)", np.nan),
            "R/R": row.get("R/R", np.nan),
            "Leverage": row["Leverage"],
            "IV": row.get("IV", np.nan),
            "Sharpe": row.get("Sharpe", np.nan),
            "Sortino": row.get("Sortino", np.nan),
            "Val Score": row.get("Val Score", 50),
            "Edge Score": row.get("Edge Score", np.nan),
            "Bounce Score": row.get("Bounce Score", 0),
            "Final Score": row.get("Final Score", np.nan),
            "Portfolio Score": row["Portfolio Score"],
            "Eff. Score": effective_score,
            "Corr Penalty": corr_penalty,
            "Breakeven %": row.get("Breakeven %", np.nan),
            "Max DD": row.get("Max DD", np.nan),
        }
    )

    conv = "⭐" if ticker in CONVICTION else "  "
    bucket_emoji = bucket.split(" ")[0]
    print(
        f"  {conv} {bucket_emoji} {ticker:5s}  ${row['Strike']:,.0f} {row['Expiry']} ({row['DTE']}d)  "
        f"×{n_contracts} = ${total_cost:,.0f}  "
        f"({total_cost / STARTING_BALANCE:.0%} of port)  "
        f"Kelly={kelly_f:.1%}  PScore={effective_score:.0f}"
    )
    return remaining, True


# ── Round-robin allocation: fill each bucket toward its target ──
MAX_ROUNDS = 6  # prevent infinite loops
for round_num in range(MAX_ROUNDS):
    if remaining < 100:
        break

    made_progress = False

    for bucket in bucket_order:
        target = bucket_targets.get(bucket, 0)
        current = bucket_allocated.get(bucket, 0)

        # Skip if bucket is already at/above target (with 20% tolerance)
        if current >= target * 1.2 and round_num < MAX_ROUNDS - 1:
            continue

        # Try adding the next best candidate from this bucket's pool
        pool_iter = bucket_pools[bucket]
        tries = 0
        while tries < 50:  # don't burn through entire pool in one pass
            tries += 1
            try:
                _, row = next(pool_iter)
            except StopIteration:
                break

            remaining, added = try_add_position(
                row,
                remaining,
                ticker_allocated,
                bucket_allocated,
                selected_tickers,
                selected_options,
                portfolio_rows,
            )
            if added:
                made_progress = True
                break

    if not made_progress:
        # Last resort: try any remaining candidate regardless of bucket
        if round_num == MAX_ROUNDS - 1:
            break

        # Re-sort all remaining candidates and try
        all_remaining = candidates[
            ~candidates.apply(
                lambda r: (r["Ticker"], r["Strike"], r["Expiry"]) in selected_options,
                axis=1,
            )
        ].sort_values("Portfolio Score", ascending=False)

        for _, row in all_remaining.head(20).iterrows():
            if remaining < 100:
                break
            remaining, added = try_add_position(
                row,
                remaining,
                ticker_allocated,
                bucket_allocated,
                selected_tickers,
                selected_options,
                portfolio_rows,
            )
            if added:
                made_progress = True

        if not made_progress:
            break

portfolio_df = pd.DataFrame(portfolio_rows)
cash_remaining = remaining

print(f"\n{'─' * 60}")
print(f"  Positions:  {len(portfolio_df)}")
print(
    f"  Tickers:    {portfolio_df['Ticker'].nunique() if not portfolio_df.empty else 0}"
)
print(
    f"  Invested:   ${STARTING_BALANCE - cash_remaining:,.0f} ({(STARTING_BALANCE - cash_remaining) / STARTING_BALANCE:.0%})"
)
print(f"  Cash:       ${cash_remaining:,.0f} ({cash_remaining / STARTING_BALANCE:.0%})")
for b in bucket_order:
    ba = bucket_allocated.get(b, 0)
    bt = bucket_targets.get(b, 0)
    print(
        f"    {b}: ${ba:,.0f} (target ${bt:,.0f}, {'✅' if ba >= bt * 0.7 else '⚠️ under'})"
    )
print(f"{'═' * 60}")


# ══════════════════════════════════════════════════════════════════
# 7.  PORTFOLIO DASHBOARD
# ══════════════════════════════════════════════════════════════════

display(Markdown("---\n# 💼 Portfolio — $15k Call Options Book\n"))

if portfolio_df.empty:
    display(Markdown("❌ No positions could be constructed. Check balance or scoring."))
else:
    # ── Holdings Table ──
    display(Markdown("### 📋 Holdings"))

    hold_cols = [
        "Ticker",
        "Conviction",
        "Bucket",
        "Expiry",
        "DTE",
        "Strike",
        "Moneyness",
        "Mid",
        "Contracts",
        "Total Cost",
        "% of Portfolio",
        "P(Profit)",
        "R/R",
        "Val Score",
        "Portfolio Score",
    ]
    ha = [c for c in hold_cols if c in portfolio_df.columns]

    display(
        portfolio_df[ha]
        .style.format(
            {
                "Strike": "${:,.0f}",
                "Mid": "${:,.2f}",
                "Moneyness": "{:.0%}",
                "Total Cost": "${:,.0f}",
                "% of Portfolio": "{:.1%}",
                "P(Profit)": "{:.0%}",
                "R/R": "{:.1f}x",
                "Val Score": "{:.0f}",
                "Portfolio Score": "{:.0f}",
            },
            na_rep="—",
        )
        .background_gradient(
            subset=["Portfolio Score"], cmap="RdYlGn", vmin=30, vmax=75
        )
        .background_gradient(subset=["% of Portfolio"], cmap="Blues", vmin=0, vmax=0.15)
        .set_caption(
            f"Portfolio holdings — ${STARTING_BALANCE - cash_remaining:,.0f} invested, ${cash_remaining:,.0f} cash"
        )
    )

    # ── Risk Summary Card ──
    display(Markdown("### 📊 Risk Summary"))

    total_invested = STARTING_BALANCE - cash_remaining
    max_loss = total_invested  # all options go to zero
    weighted_pp = (
        (
            (portfolio_df["P(Profit)"].fillna(0) * portfolio_df["Total Cost"]).sum()
            / total_invested
        )
        if total_invested > 0
        else 0
    )
    weighted_rr = (
        (
            (portfolio_df["R/R"].fillna(0) * portfolio_df["Total Cost"]).sum()
            / total_invested
        )
        if total_invested > 0
        else 0
    )
    avg_dte = (
        ((portfolio_df["DTE"] * portfolio_df["Total Cost"]).sum() / total_invested)
        if total_invested > 0
        else 0
    )
    n_tickers = portfolio_df["Ticker"].nunique()
    n_positions = len(portfolio_df)
    avg_leverage = (
        ((portfolio_df["Leverage"] * portfolio_df["Total Cost"]).sum() / total_invested)
        if total_invested > 0
        else 0
    )

    # Herfindahl index for concentration
    weights = portfolio_df["Total Cost"] / total_invested
    hhi = (weights**2).sum()
    effective_positions = 1 / hhi if hhi > 0 else 0

    risk_data = {
        "Metric": [
            "Starting Balance",
            "Total Invested",
            "Cash Reserve",
            "Max Loss (all expire worthless)",
            "Positions / Tickers",
            "Weighted P(Profit)",
            "Weighted R/R @+10%",
            "Weighted Avg DTE",
            "Weighted Avg Leverage",
            "Concentration (HHI)",
            "Effective # Positions",
        ],
        "Value": [
            f"${STARTING_BALANCE:,.0f}",
            f"${total_invested:,.0f} ({total_invested / STARTING_BALANCE:.0%})",
            f"${cash_remaining:,.0f} ({cash_remaining / STARTING_BALANCE:.0%})",
            f"${max_loss:,.0f} ({max_loss / STARTING_BALANCE:.0%})",
            f"{n_positions} positions across {n_tickers} tickers",
            f"{weighted_pp:.0%}",
            f"{weighted_rr:.1f}x",
            f"{avg_dte:.0f} days",
            f"{avg_leverage:.0f}x",
            f"{hhi:.2f} (1.0 = all in one, lower = diversified)",
            f"{effective_positions:.1f}",
        ],
    }
    display(
        pd.DataFrame(risk_data)
        .style.hide(axis="index")
        .set_caption("Portfolio risk metrics")
    )

    # ── Allocation by Bucket — Pie Chart ──
    display(Markdown("### 🥧 Allocation Breakdown"))

    bucket_alloc = portfolio_df.groupby("Bucket")["Total Cost"].sum().reset_index()
    bucket_alloc["Pct"] = bucket_alloc["Total Cost"] / total_invested
    # Add cash
    bucket_alloc = pd.concat(
        [
            bucket_alloc,
            pd.DataFrame(
                [
                    {
                        "Bucket": "💵 Cash",
                        "Total Cost": cash_remaining,
                        "Pct": cash_remaining / STARTING_BALANCE,
                    }
                ]
            ),
        ],
        ignore_index=True,
    )

    fig_pie_bucket = px.pie(
        bucket_alloc,
        values="Total Cost",
        names="Bucket",
        title="Allocation by Time Horizon",
        color="Bucket",
        color_discrete_map={
            "⚡ <30 DTE": "#e74c3c",
            "🔄 30-90 DTE": "#f39c12",
            "🏗️ 300+ DTE": "#2ecc71",
            "💵 Cash": "#95a5a6",
        },
        hole=0.35,
    )
    fig_pie_bucket.update_traces(textinfo="label+percent", textposition="outside")
    fig_pie_bucket.show()

    # Allocation by Ticker
    ticker_alloc = portfolio_df.groupby("Ticker")["Total Cost"].sum().reset_index()
    ticker_alloc = pd.concat(
        [
            ticker_alloc,
            pd.DataFrame([{"Ticker": "Cash", "Total Cost": cash_remaining}]),
        ],
        ignore_index=True,
    )

    fig_pie_ticker = px.pie(
        ticker_alloc,
        values="Total Cost",
        names="Ticker",
        title="Allocation by Ticker",
        hole=0.35,
    )
    fig_pie_ticker.update_traces(textinfo="label+percent", textposition="outside")
    fig_pie_ticker.show()

    # ── Scenario P&L — Portfolio Level ──
    display(Markdown("### 💰 Scenario P&L — Portfolio Level"))
    display(Markdown("*What happens to the whole book if the market moves uniformly.*"))

    scenarios = [-0.15, -0.10, -0.05, 0.0, 0.05, 0.10, 0.15, 0.20, 0.30]
    scen_rows = []

    for move in scenarios:
        total_payout = 0
        for _, pos in portfolio_df.iterrows():
            new_spot = pos["Spot"] * (1 + move)
            intrinsic = max(new_spot - pos["Strike"], 0)
            payout = intrinsic * 100 * pos["Contracts"]
            total_payout += payout

        total_cost = total_invested
        pnl = total_payout - total_cost
        pnl_pct = pnl / STARTING_BALANCE  # P&L as % of full balance

        scen_rows.append(
            {
                "Market Move": f"{move:+.0%}",
                "Portfolio Value": total_payout,
                "P&L ($)": pnl,
                "P&L (% of Balance)": pnl_pct,
                "Total Return": (total_payout + cash_remaining) / STARTING_BALANCE - 1,
            }
        )

    scen_pnl = pd.DataFrame(scen_rows)
    display(
        scen_pnl.style.format(
            {
                "Portfolio Value": "${:,.0f}",
                "P&L ($)": "${:+,.0f}",
                "P&L (% of Balance)": "{:+.1%}",
                "Total Return": "{:+.1%}",
            }
        )
        .background_gradient(
            subset=["P&L ($)"],
            cmap="RdYlGn",
            vmin=-total_invested,
            vmax=total_invested * 2,
        )
        .hide(axis="index")
        .set_caption(
            f"At-expiry intrinsic value — starting balance ${STARTING_BALANCE:,.0f}"
        )
    )

    # ── P&L waterfall chart ──
    fig_pnl = go.Figure()
    fig_pnl.add_trace(
        go.Bar(
            x=scen_pnl["Market Move"],
            y=scen_pnl["P&L ($)"],
            marker_color=[
                "#e74c3c" if v < 0 else "#2ecc71" for v in scen_pnl["P&L ($)"]
            ],
            text=[f"${v:+,.0f}" for v in scen_pnl["P&L ($)"]],
            textposition="outside",
        )
    )
    fig_pnl.update_layout(
        title="Portfolio P&L by Market Move (at expiry)",
        xaxis_title="Uniform Market Move",
        yaxis_title="P&L ($)",
        height=400,
        showlegend=False,
    )
    fig_pnl.add_hline(y=0, line_dash="dot", line_color="gray")
    fig_pnl.show()

    # ── Per-Position Scenario Detail ──
    display(Markdown("### 📊 Per-Position Return Scenarios"))

    pos_scen_rows = []
    for _, pos in portfolio_df.iterrows():
        for move in [-0.05, 0.05, 0.10, 0.20]:
            new_spot = pos["Spot"] * (1 + move)
            intrinsic = max(new_spot - pos["Strike"], 0)
            pos_pnl = (intrinsic - pos["Mid"]) * 100 * pos["Contracts"]
            pos_ret = (intrinsic - pos["Mid"]) / pos["Mid"] if pos["Mid"] > 0 else 0
            pos_scen_rows.append(
                {
                    "Ticker": pos["Ticker"],
                    "Bucket": pos["Bucket"],
                    "Strike": f"${pos['Strike']:,.0f}",
                    "Cost": f"${pos['Total Cost']:,.0f}",
                    "Move": move,
                    "Return": pos_ret,
                }
            )

    pos_scen_df = pd.DataFrame(pos_scen_rows)
    if not pos_scen_df.empty:
        pos_piv = pos_scen_df.pivot_table(
            index=["Bucket", "Ticker", "Strike", "Cost"],
            columns="Move",
            values="Return",
        )
        pos_piv.columns = [f"{c:+.0%}" for c in pos_piv.columns]
        display(
            pos_piv.style.format("{:+.0%}", na_rep="—")
            .background_gradient(cmap="RdYlGn", vmin=-1.0, vmax=3.0, axis=None)
            .set_caption("Per-position return at expiry")
        )

    # ── Correlation Heatmap of Selected Positions ──
    display(Markdown("### 🔗 Correlation — Selected Tickers"))

    port_tickers = sorted(portfolio_df["Ticker"].unique().tolist())
    port_tickers_in_corr = [t for t in port_tickers if t in corr_matrix.columns]

    if len(port_tickers_in_corr) > 1:
        sub_corr = corr_matrix.loc[port_tickers_in_corr, port_tickers_in_corr]

        fig_corr = go.Figure(
            data=go.Heatmap(
                z=sub_corr.values,
                x=sub_corr.columns.tolist(),
                y=sub_corr.index.tolist(),
                colorscale="RdYlGn_r",
                zmin=-0.2,
                zmax=1.0,
                text=np.round(sub_corr.values, 2),
                texttemplate="%{text}",
                textfont={"size": 12},
            )
        )
        fig_corr.update_layout(
            title="Pairwise Correlation — Portfolio Tickers (6M daily returns)",
            height=400,
            width=500,
        )
        fig_corr.show()

        # Flag highly correlated pairs
        high_corr_pairs = []
        for i, t1 in enumerate(port_tickers_in_corr):
            for j, t2 in enumerate(port_tickers_in_corr):
                if i < j and sub_corr.loc[t1, t2] > 0.65:
                    high_corr_pairs.append((t1, t2, sub_corr.loc[t1, t2]))

        if high_corr_pairs:
            display(Markdown("**⚠️ Correlated pairs in portfolio:**"))
            for t1, t2, rho in sorted(high_corr_pairs, key=lambda x: -x[2]):
                display(
                    Markdown(
                        f"- {t1} ↔ {t2}: ρ = {rho:.2f} — consider these as overlapping exposure"
                    )
                )
        else:
            display(
                Markdown(
                    "✅ No highly correlated pairs (ρ > 0.65) — good diversification."
                )
            )

    # ── Position Sizing Breakdown ──
    display(Markdown("### 🎯 Sizing Logic Breakdown"))

    sizing_cols = [
        "Ticker",
        "Bucket",
        "Strike",
        "DTE",
        "Kelly Frac",
        "Contracts",
        "Total Cost",
        "% of Portfolio",
        "P(Profit)",
        "R/R",
        "Corr Penalty",
        "Portfolio Score",
    ]
    sa = [c for c in sizing_cols if c in portfolio_df.columns]
    display(
        portfolio_df[sa]
        .style.format(
            {
                "Strike": "${:,.0f}",
                "Kelly Frac": "{:.1%}",
                "Total Cost": "${:,.0f}",
                "% of Portfolio": "{:.1%}",
                "P(Profit)": "{:.0%}",
                "R/R": "{:.1f}x",
                "Corr Penalty": "{:.2f}",
                "Portfolio Score": "{:.0f}",
            },
            na_rep="—",
        )
        .set_caption(
            "How each position was sized — Kelly fraction × balance constraints"
        )
    )

    # ── Final Summary ──
    display(Markdown("---"))
    display(
        Markdown(
            f"### ✅ Portfolio Complete\n\n"
            f"- **{n_positions} positions** across **{n_tickers} tickers** and "
            f"**{portfolio_df['Bucket'].nunique()} time horizons**\n"
            f"- **${total_invested:,.0f} deployed** ({total_invested / STARTING_BALANCE:.0%}) "
            f"with **${cash_remaining:,.0f} cash reserve** ({cash_remaining / STARTING_BALANCE:.0%})\n"
            f"- Weighted P(Profit): **{weighted_pp:.0%}** | Weighted R/R: **{weighted_rr:.1f}x** "
            f"| Avg DTE: **{avg_dte:.0f}d**\n"
            f"- Effective positions: **{effective_positions:.1f}** (HHI = {hhi:.2f})\n\n"
            f"**Portfolio is max-loss capped at ${max_loss:,.0f}** (options expire worthless). "
            f"Cash reserve provides dry powder for dips or rolling positions.\n\n"
            f"*Rebalance triggers:* Roll LEAPS when DTE < 90. "
            f"Take profit on swings at 2× entry. "
            f"Re-run this notebook weekly for updated scores."
        )
    )

Fetching fundamentals (P/E, PEG, FCF, market cap)…

  WDC    FwdPE=21.6      PEG=0.88    FCFy=4.0%   MCap=$97B
  GEV    FwdPE=35.1      PEG=3.17    FCFy=2.5%   MCap=$211B
  STX    FwdPE=22.2      PEG=0.92    FCFy=1.2%   MCap=$94B
  LRCX   FwdPE=34.0      PEG=2.16    FCFy=1.7%   MCap=$290B
  AMAT   FwdPE=26.5      PEG=2.72    FCFy=1.4%   MCap=$256B
  TSM    FwdPE=19.4      PEG=1.23    FCFy=34.2%  MCap=$1809B
  GE     FwdPE=37.8      PEG=5.55    FCFy=1.5%   MCap=$339B
  CMI    FwdPE=18.4      PEG=2.39    FCFy=1.7%   MCap=$80B
  KLAC   FwdPE=31.6      PEG=2.26    FCFy=1.7%   MCap=$190B
  SNPS   FwdPE=25.3      PEG=2.68    FCFy=2.7%   MCap=$82B
  META   FwdPE=18.7      PEG=1.19    FCFy=1.4%   MCap=$1673B
  UBER   FwdPE=17.5      PEG=7.74    FCFy=4.2%   MCap=$156B
  ISRG   FwdPE=42.7      PEG=2.55    FCFy=1.3%   MCap=$173B
  MSFT   FwdPE=21.3      PEG=1.53    FCFy=1.8%   MCap=$2981B
  AMZN   FwdPE=22.7      PEG=1.59    FCFy=1.1%   MCap=$2258B
  AVGO   FwdPE=23.2      PEG=0.93    FCFy=1.6%  

### 📊 Valuation Scores

,Ticker,Val Score,Fwd PE,PEG,FCF Yield,Rev Growth,Profit Margin
5,TSM,74,19.4,1.23,34.2%,20.5%,45.1%
0,WDC,74,21.6,0.88,4.0%,—,35.6%
10,META,68,18.7,1.19,1.4%,23.8%,30.1%
15,AVGO,64,23.2,0.93,1.6%,16.4%,36.2%
2,STX,61,22.2,0.92,1.2%,21.5%,19.6%
13,MSFT,59,21.3,1.53,1.8%,16.7%,39.0%
11,UBER,59,17.5,7.74,4.2%,20.1%,19.3%
3,LRCX,55,34.0,2.16,1.7%,22.1%,30.2%
14,AMZN,52,22.7,1.59,1.1%,13.6%,10.8%
9,SNPS,52,25.3,2.68,2.7%,37.8%,18.9%



Building correlation matrix…
✅ Correlation matrix: 16×16

════════════════════════════════════════════════════════════
  💼  BUILDING PORTFOLIO — Round-Robin Bucket Builder
════════════════════════════════════════════════════════════
  Starting balance: $15,000
  Kelly fraction:   25% of full Kelly
  Edge haircut:     90% of BSM P(Profit)
  Max per ticker:   25%
  Max per position: 20%
  Bucket targets:   LEAPS 55% / Med 30% / Short 15%

     🏗️ TSM    $340 2026-05-15 (96d)  ×1 = $3,572  (24% of port)  Kelly=0.5%  PScore=68
     🔄 LRCX   $210 2026-03-20 (40d)  ×1 = $3,075  (20% of port)  Kelly=0.5%  PScore=55
     ⚡ CMI    $590 2026-02-20 (12d)  ×1 = $590  (4% of port)  Kelly=3.4%  PScore=57
     🔄 WDC    $270 2026-03-13 (33d)  ×1 = $3,575  (24% of port)  Kelly=0.5%  PScore=57
     ⚡ CMI    $580 2026-02-20 (12d)  ×1 = $1,070  (7% of port)  Kelly=3.5%  PScore=26
     ⚡ AMZN   $220 2026-02-20 (12d)  ×1 = $192  (1% of port)  Kelly=0.0%  PScore=53
     🏗️ GE     $310 2026-05-15 (96d)  ×1 =

---
# 💼 Portfolio — $15k Call Options Book


...

### 🎯 Sizing Logic Breakdown

,Ticker,Bucket,Strike,DTE,Kelly Frac,Contracts,Total Cost,% of Portfolio,P(Profit),R/R,Corr Penalty,Portfolio Score
0,TSM,🏗️ 300+ DTE,$340,96,0.5%,1,"$3,572",23.8%,35%,0.2x,1.00,68
1,LRCX,🔄 30-90 DTE,$210,40,0.5%,1,"$3,075",20.5%,39%,0.4x,0.97,57
2,CMI,⚡ <30 DTE,$590,12,3.4%,1,$590,3.9%,27%,6.7x,1.00,57
3,WDC,🔄 30-90 DTE,$270,33,0.5%,1,"$3,575",23.8%,34%,0.1x,1.00,57
4,CMI,⚡ <30 DTE,$580,12,3.5%,1,"$1,070",7.1%,34%,4.2x,0.48,54
5,AMZN,⚡ <30 DTE,$220,12,0.0%,1,$192,1.3%,19%,4.9x,1.00,53
6,GE,🏗️ 300+ DTE,$310,96,0.5%,1,"$2,855",19.0%,38%,0.5x,1.00,49


---

### ✅ Portfolio Complete

- **7 positions** across **6 tickers** and **3 time horizons**
- **$14,930 deployed** (100%) with **$70 cash reserve** (0%)
- Weighted P(Profit): **35%** | Weighted R/R: **0.9x** | Avg DTE: **59d**
- Effective positions: **5.0** (HHI = 0.20)

**Portfolio is max-loss capped at $14,930** (options expire worthless). Cash reserve provides dry powder for dips or rolling positions.

*Rebalance triggers:* Roll LEAPS when DTE < 90. Take profit on swings at 2× entry. Re-run this notebook weekly for updated scores.

: 